# Objetos Fundamentales

Para ejecutar todos los ejemplos se debe importar la librería. Se sugiere utilizar siempre el alias `qcf`. 

In [1]:
import qcfinancial as qcf
import pandas as pd

## Monedas

Las divisas se representan con objetos de tipo `QCCurrency` y sus subclases. En estos momentos, de forma explícita, están implementadas las siguientes divisas:


|  Lista | de  | Divisas  |
|---|---|---|
| AUD  | CNY  | JPY   |
| BRL  | COP  | MXN  |
| CAD   | DKK   | NOK   |
| CHF  | EUR   | PEN  |
| CLF  | GBP   | SEK  |
| CLP  | HKD   | USD  |

Si se requiere otra, solicitarlo ingresando un *issue* en el [git repo](https://github.com/qcfinancial/qcfinancial.git) del proyecto.

El constructor por default retorna USD.

In [2]:
x = qcf.QCCurrency()
x.get_iso_code()

'USD'

Alta de divisas CLP, USD y JPY (USD se puede instanciar también de forma explícita).

In [3]:
monedas = [
    clp:=qcf.QCCLP(),
    usd:=qcf.QCUSD(),
    jpy:=qcf.QCJPY(),
]

Sin usar las subclases se puede instanciar una divisa que no esté implementada, por ejemplo para el peso argentino ARS:

In [4]:
ars = qcf.QCCurrency('Peso argentino', 'ARS', 32, 0)
monedas.append(ars)

### Métodos: `get_name`, `get_iso_code`, `get_iso_number`, `get_decimal_places` y `amount`.

El método `amount`debe utilizarse cuando se debe pagar o recibir un monto resultado de un cálculo. De esta forma, el monto se redondea al número correcto de decimales en la divisa (que se obtiene con `get_decimal_places`). Por ejemplo, en CLP, se redondea a 0 decimales ya que en esta divisa no se utilizan los centavos.

In [5]:
cantidad = 100.123456
for moneda in monedas:
    print(f"Nombre: {format(moneda.get_name())}")
    print(f"Código ISO: {moneda.get_iso_code()}")
    print(f"Número ISO: {moneda.get_iso_number()}")
    print(f"Número de decimales: {moneda.get_decimal_places()}")
    print(f"Cantidad {cantidad} con el número correcto de decimales: {moneda.amount(cantidad):.4f}")
    print()

Nombre: Chilean Peso
Código ISO: CLP
Número ISO: 152
Número de decimales: 0
Cantidad 100.123456 con el número correcto de decimales: 100.0000

Nombre: U. S. Dollar
Código ISO: USD
Número ISO: 840
Número de decimales: 2
Cantidad 100.123456 con el número correcto de decimales: 100.1200

Nombre: Japanese Yen
Código ISO: JPY
Número ISO: 392
Número de decimales: 2
Cantidad 100.123456 con el número correcto de decimales: 100.1200

Nombre: Peso argentino
Código ISO: ARS
Número ISO: 32
Número de decimales: 0
Cantidad 100.123456 con el número correcto de decimales: 100.0000



Más adelante, cuando veamos el concepto de índice, veremos como disponer de un objeto moneda simplifica la conversión de montos de una moneda a otra.

## Fechas

Las fechas se representan con objetos de tipo `QCDate`. Para inicializar un `QCDate` se requiere el día, el mes y el año de la fecha. También se puede inicializar sin valor (default constructor) en cuyo caso se obtendrá el 12-01-1969.

### Constructores

Inicializar sin valor. Se muestra además los métodos `description()` y `__str__()`.

In [6]:
fecha = qcf.QCDate()
print(f"description(True): {fecha.description(True)}")
print(f"description(False): {fecha.description(False)}")
print(f"iso_code(): {fecha.iso_code()}")
print(f"__str__: {fecha}")

description(True): 12-01-1969
description(False): 1969-01-12
iso_code(): 1969-01-12
__str__: 1969-01-12


Inicializar con una fecha específica. En este caso, el contructor realiza una validación de los parámetros iniciales.

In [7]:
fecha1 = qcf.QCDate(14, 9, 2024)  # día, mes, año
print(f"Fecha: {fecha1}")

Fecha: 2024-09-14


Error al tratar de construir una fecha inválida.

In [8]:
try:
    fecha0 = qcf.QCDate(31, 2, 2024)  # ¡¡¡ 31 de febrero !!!
except ValueError as e:
    print(e)

Invalid day for month = 2


### Getters y Setters

Métodos: `set_day`, `set_month` y `set_year`.

In [9]:
fecha1.set_day(17)
fecha1.set_month(10)
fecha1.set_year(2024)
print(f"Fecha: {fecha1}")

Fecha: 2024-10-17


Métodos `day`, `month` y `year`.

In [10]:
print(f"Día: {fecha1.day()}")
print(f"Mes: {fecha1.month()}")
print(f"Año: {fecha1.year()}")

Día: 17
Mes: 10
Año: 2024


### Método `week_day`

Retorna una variable de tipo `enum QC_Financial.WeekDay` que representa el día de la semana al que corresponde a la fecha.

In [11]:
dia_semana = fecha1.week_day()
print(f"Tipo del retorno: {type(dia_semana)}")
print(f"Día de la semana: {dia_semana}")

Tipo del retorno: <class 'qcfinancial.WeekDay'>
Día de la semana: WeekDay.THU


### Método `add_months`

Suma **n meses** a `fecha1` y retorna esa nueva fecha sin cambiar el valor de `fecha1`.

In [12]:
num_meses = 1
fecha2 = fecha1.add_months(num_meses)
print(f"fecha1: {fecha1}")
print(f"fecha2: {fecha2}")

fecha1: 2024-10-17
fecha2: 2024-11-17


### Método `add_days`

Suma **n días** a `fecha1` sin cambiar el valor de `fecha1`.

In [13]:
num_dias = 30
fecha3 = fecha1.add_days(num_dias)
print(f"fecha1: {fecha1}")
print(f"fecha3: {fecha3}")

fecha1: 2024-10-17
fecha3: 2024-11-16


### Método `day_diff`

Calcula la diferencia en días con otra fecha. Si la otra fecha es mayor el resultado es positivo, si no, es negativo.

In [14]:
# Dado que fecha3 > fecha1 entonces el resultado es positivo
print(f"fecha1.day_diff(fecha3): {fecha1.day_diff(fecha3)} (días)")

# Se invierten los roles y el resultado es negativo
print(f"fecha3.day_diff(fecha1): {fecha3.day_diff(fecha1)} (días)")

fecha1.day_diff(fecha3): 30 (días)
fecha3.day_diff(fecha1): -30 (días)


### Orden en `QCDate`

El orden de `QCDate` permite que las fechas pueden compararse entre si.

In [15]:
print(f"fecha1: {fecha1}")
print(f"fecha2: {fecha2}")
print(f"fecha1 == fecha2: {fecha1 == fecha2}")
print(f"fecha1 != fecha2: {fecha1 != fecha2}")
print(f"fecha1 < fecha2: {fecha1 < fecha2}")
print(f"fecha1 <= fecha2: {fecha1 <= fecha2}")
print(f"fecha1 > fecha2: {fecha1 > fecha2}")
print(f"fecha1 >= fecha2: {fecha1 >= fecha2}")

fecha1: 2024-10-17
fecha2: 2024-11-17
fecha1 == fecha2: False
fecha1 != fecha2: True
fecha1 < fecha2: True
fecha1 <= fecha2: True
fecha1 > fecha2: False
fecha1 >= fecha2: False


### Un objeto `QCDate` es *hashable*

Esto permite que las fechas puedan usarse como `key` en un `dict`de Python. El hash que se utiliza coincide con la representación como entero de una fecha que se utiliza en Excel.

In [16]:
print(fecha1.__hash__())

45582


Por ejemplo, una serie de tiempo:

In [17]:
serie_de_tiempo = {
    qcf.QCDate(22, 5, 2024): 100.01,
    qcf.QCDate(23, 5, 2024): 100.02,
    qcf.QCDate(24, 5, 2024): 100.03,
}
print(f"Valor al: {qcf.QCDate(23, 5, 2024)} es {serie_de_tiempo[qcf.QCDate(23, 5, 2024)]}")

Valor al: 2024-05-23 es 100.02


### Método `build_qcdate_from_string`

Se trata de un *factory method* que permite inicializar un objeto `QCDate` a partir de un `string`.
El formato del `string` debe ser yyyy&mm&dd donde & es un separador arbitrario.

In [18]:
str1 = "2020-01-01"
str2 = "2020/01/02"
str3 = "2020&01&03"

fecha4 = qcf.build_qcdate_from_string(str1)
print(f"{str1}: {fecha4}")

fecha4 = qcf.build_qcdate_from_string(str2)
print(f"{str2}: {fecha4}")

fecha4 = qcf.build_qcdate_from_string(str3)
print(f"{str3}: {fecha4}")

2020-01-01: 2020-01-01
2020/01/02: 2020-01-02
2020&01&03: 2020-01-03


## Calendarios
Los calendarios se representan con objetos de tipo `BusinesssCalendar` y son **listas** de fechas arbitrarias que representan días feriados en alguna ciudad, país, región o unión de las anteriores. Para dar de alta un calendario se requiere una fecha inicial (`QCDate` y un número entero positivo que representa el plazo inicial total del calendario en años). El objeto `BusinessCalendar` incluye explícitamente todos los días 1 de enero y considera siempre como feriado los días sábado y domingo (aunque no pertenecen de forma explícita a la **lista**).

En el siguiente loop, por ejemplo, no se imprime nada.

In [19]:
scl = qcf.BusinessCalendar(fecha1, 10)
for holiday in scl.get_holidays():
    print(holiday)

### Método `add_holiday`

Agrega una fecha a la lista.

In [20]:
scl.add_holiday(qcf.QCDate(2, 1, 2018))
for holiday in scl.get_holidays():
    print(holiday)

2018-01-02


### Método `next_busy_day`

Dada una fecha, si ésta es hábil retorna la misma fecha, si, por el contrario, la fecha es inhábil, devuelve la siguiente fecha hábil del calendario.

In [21]:
print("next:", scl.next_busy_day(qcf.QCDate(15, 9, 2018)))  # es sábado
print("Se agrega el 17-9-2018 a la lista")
scl.add_holiday(qcf.QCDate(17, 9, 2018))
print("nuevo next:", scl.next_busy_day(qcf.QCDate(15, 9, 2018)))

next: 2018-09-17
Se agrega el 17-9-2018 a la lista
nuevo next: 2018-09-18


### Método `mod_next_busy_day`

Opera igual que la función anterior a menos que la función anterior retorne una fecha del mes siguiente, en ese caso retorna la fecha hábil anterior.

In [22]:
abril30 = qcf.QCDate(30, 4, 2024)
print(f"Abril 30: {abril30.week_day()}, {abril30}")

scl.add_holiday(abril30)

print("next:", scl.next_busy_day(abril30))
print("mod next:", scl.mod_next_busy_day(abril30))

Abril 30: WeekDay.TUE, 2024-04-30
next: 2024-05-01
mod next: 2024-04-29


### Método `prev_busy_day`

Opera de forma análoga a `busy_day`, pero retornando la fecha hábil anterior.

In [23]:
print("prev:", scl.prev_busy_day(abril30))

prev: 2024-04-29


### Método `shift`

Suma un número *n* (positivo o negativo) de días hábiles a una fecha.

In [24]:
mayo2 = qcf.QCDate(2, 5, 2024)
print(scl.shift(mayo2, -1))
print(scl.shift(abril30, 0))
print(scl.shift(abril30, 1))
print(scl.shift(abril30, 5))

2024-05-01
2024-04-30
2024-05-01
2024-05-07


Agreguemos el 2024-05-01 a los feriados de `scl` y veamos cómo cambia el primer resultado.

In [25]:
scl.add_holiday(qcf.QCDate(1, 5, 2024))
print(scl.shift(mayo2, -1))

2024-04-29


Se va al 29 de abril porque también agregamos como feriado el 30 de abril.

## Fracciones de Año

Las fracciones de año corresponden a las distintas formas de medir un intervalo de tiempo entre dos fechas que comunmente se utilizan en los productos de tasa de interés.

In [26]:
yfs = [
    act360:=qcf.QCAct360(),
    act365:=qcf.QCAct365(),
    act30:=qcf.QCAct30(),
    t30360:=qcf.QC30360(),
    t3030:=qcf.QC3030(),
    
    # Corresponde a depósitos a plazo en CLP
    act30:=qcf.QCAct30(),

    # La utilizan los bonos del tesoro americano
    actact:=qcf.QCActAct(),
]

### Métodos `yf` y `count_days`

El método `yf`, que retorna el valor de la fracción de año, está sobrecargado, se puede calcular usando como argumentos un número de días o un par de fechas (`QCDate`).

In [27]:
print(f"\nfecha1: {fecha1} y fecha3: {fecha3}")
print("---------------------------------------\n")
for yf in yfs:
    print(type(yf))
    print(f"yf(30): {yf.yf(30):.6f}")
    print(f"yf.yf(fecha1, fecha3): {yf.yf(fecha1, fecha3):.6f}")
    print(f"yf.yf.count_days(fecha1, fecha3): {yf.count_days(fecha1, fecha3):.0f}")
    print()


fecha1: 2024-10-17 y fecha3: 2024-11-16
---------------------------------------

<class 'qcfinancial.QCAct360'>
yf(30): 0.083333
yf.yf(fecha1, fecha3): 0.083333
yf.yf.count_days(fecha1, fecha3): 30

<class 'qcfinancial.QCAct365'>
yf(30): 0.082192
yf.yf(fecha1, fecha3): 0.082192
yf.yf.count_days(fecha1, fecha3): 30

<class 'qcfinancial.QCAct30'>
yf(30): 1.000000
yf.yf(fecha1, fecha3): 1.000000
yf.yf.count_days(fecha1, fecha3): 30

<class 'qcfinancial.QC30360'>
yf(30): 0.083333
yf.yf(fecha1, fecha3): 0.080556
yf.yf.count_days(fecha1, fecha3): 29

<class 'qcfinancial.QC3030'>
yf(30): 1.000000
yf.yf(fecha1, fecha3): 0.966667
yf.yf.count_days(fecha1, fecha3): 29

<class 'qcfinancial.QCAct30'>
yf(30): 1.000000
yf.yf(fecha1, fecha3): 1.000000
yf.yf.count_days(fecha1, fecha3): 30

<class 'qcfinancial.QCActAct'>
yf(30): 0.082192
yf.yf(fecha1, fecha3): 0.082192
yf.yf.count_days(fecha1, fecha3): 30



## Funciones y Factores de Capitalización

Las funciones de capitalización representan las distintas formas en que se puede usar el valor de una tasa de interés para calcular o traer a valor presente un flujo de caja futuro. Al resultado de la función de capitalización lo llamamos *factor de capitalización*.

Están disponibles los siguientes 3 tipos de funciones (donde $yf$ es la fracción de año asociada a la tasa de valor $r$):
- QCLinearWf     ---> $1 + r \cdot yf$
- QCCompundWf    ---> $1 + r \cdot yf$
- QCContinousWf  ---> $exp(r \cdot yf)$

In [28]:
wfs = [
    lin_wf:=qcf.QCLinearWf(),
    com_wf:=qcf.QCCompoundWf(),
    exp_wf:=qcf.QCContinousWf(),
]

### Método `wf`

Este método permite calcular el factor de capitalización a partir del valor de una tasa y el valor de una fracción de año.

In [29]:
r = .1   # Valor de la tasa
yf = .5  # Fracción de año

for wf in wfs:
    print(f"Función: {wf}. Factor: {wf.wf(r, yf):6f}")

Función: Lin. Factor: 1.050000
Función: Com. Factor: 1.048809
Función: Exp. Factor: 1.051271


## Tasas de Interés

Utilizando un número real, una fracción de año y un factor de capitalización, se puede dar de alta (instanciar) un objeto de tipo `QCInterestRate` que representa una tasa de interés (ver por ejemplo el video [Convenciones de Tasas](https://youtu.be/AdCMPKBFwgg?si=8v4wT1WER_poqEBg)).

In [30]:
r0 = 0.1
tasas = [
    tasa_lin_act360:=qcf.QCInterestRate(0.1, act360, lin_wf),
    tasa_com_act365:= qcf.QCInterestRate(0.1, act365, com_wf),
    tasa_exp_act365:=qcf.QCInterestRate(0.1, act365, exp_wf),
]

### Métodos `get_value` y `set_value`

Permiten obtener y definir el valor de la tasa de interés.

In [31]:
for tasa in tasas:
    print(f"Descripción: {tasa}") # Está definido el método __str__
    
    # Se obtiene el valor de la tasa utilizando get_value
    print("Obtener valor:", tasa.get_value())
    
    # Se utiliza set_value para cambiar el valor de la tasa
    r1 = 0.12
    tasa.set_value(r1)
    print("Obtener nuevo valor:", tasa.get_value())
    print()

Descripción: 0.100000 Act360 Lin
Obtener valor: 0.1
Obtener nuevo valor: 0.12

Descripción: 0.100000 Act365 Com
Obtener valor: 0.1
Obtener nuevo valor: 0.12

Descripción: 0.100000 Act365 Exp
Obtener valor: 0.1
Obtener nuevo valor: 0.12



### Métodos `wf` y `dwf`

Tanto `wf` como `dwf`son métodos sobrecargados. El primero permite calcular el valor del factor de capitalización de la tasa de interés utilizando un número de días o un par de fechas, mientras que el segundo calcula la derivada del factor de capitalización respecto a la tasa de interés. 

¿Cómo se realiza el cálculo de la derivada? Veamos un ejemplo:

Consideremos una tasa de interés cuya función de capitalización es $g$. De ese modo el factor de capitalización $wf$ para un valor de tasa $r$ y una fracción de año $yf$ está dado por:

$$wf = g\left(r,yf\right)$$

En muchas situaciones nos interesará saber como cambia el factor de capitalización cuando el valor $r$ de la tasa cambia. Cuando el cambio de valor, es pequeño, digamos un punto básico, resulta conveniente calcular el cambio de valor en $wf$, $\Delta wf$ usando la derivada de la función $g$ respecto a $r$, más precisamente:

$$\Delta wf = \frac{dg\left(r,yf\right)}{dr}\left(r_0,yf\right)\cdot\delta$$

Donde $r_0$ es el valor inicial de la tasa y $\delta$ es el cambio en su valor. Tenemos que:

- Si $g=1+r\cdot yf$ entonces $\Delta wf = yf\cdot\delta$
  
- Si $g=\left(1+r\right)^{yf}$ entonces $\Delta wf = yf\cdot\left(1+r_0\right)^{yf-1}\cdot\delta$

- Si $g=exp\left(r\cdot yf\right)$ entonces $\Delta wf = yf\cdot exp\left(r_0\cdot yf\right)\cdot\delta$


Calculemos `wf` y `dwf` usando un par de fechas.

In [32]:
for i, tasa in enumerate(tasas):
    # Retorna el factor de capitalización entre las fechas
    print(f"wf(fecha1, fecha3): {tasa.wf(fecha1, fecha3):.8F}")

    # Retorna la derivada del factor de capitalización respecto al valor de la tasa entre las fechas
    print(f"dwf(fecha1, fecha3): {tasa.dwf(fecha1, fecha3):.8f}")

    # Para verificar se calcula "a mano" la derivada
    match i:
        case 0:
            print(f"Check: {tasa.yf(fecha1, fecha3) * r0:.8f}")
        case 1:
            yf_ = tasa.yf(fecha1, fecha3)
            print(f"Check: {tasa.yf(fecha1, fecha3) * (1 + r0)**(yf_ - 1):.8f}")
        case 2:
            print(f"Check: {tasa.yf(fecha1, fecha3) * tasa.wf(fecha1, fecha3):.8f}")
    
    print()

wf(fecha1, fecha3): 1.01000000
dwf(fecha1, fecha3): 0.08333333
Check: 0.00833333

wf(fecha1, fecha3): 1.00935820
dwf(fecha1, fecha3): 0.07407228
Check: 0.07530743

wf(fecha1, fecha3): 1.00991181
dwf(fecha1, fecha3): 0.08300645
Check: 0.08300645



Veamos ahora la sobrecarga y utilicemos un número de días.

In [33]:
dias = 400
for i, tasa in enumerate(tasas):
    # Retorna el factor de capitalización entre las fechas
    print(f"wf(dias): {tasa.wf(dias):.8F}")

    # Retorna la derivada del factor de capitalización respecto al valor de la tasa entre las fechas
    print(f"dwf(dias): {tasa.dwf(dias):.8f}")

    # Para verificar se calcula "a mano" la derivada
    match i:
        case 0:
            print(f"Check: {dias / 360 * r0:.8f}")
        case 1:
            yf_ = dias / 365
            print(f"Check: {yf_ * (1 + r0)**(yf_ - 1):.8f}")
        case 2:
            print(f"Check: {dias / 365 * tasa.wf(dias):.8f}")
    
    print()

wf(dias): 1.13333333
dwf(dias): 1.11111111
Check: 0.11111111

wf(dias): 1.13223756
dwf(dias): 1.10786454
Check: 1.10595203

wf(dias): 1.14054572
dwf(dias): 1.24991312
Check: 1.24991312



### Método `get_rate_from_wf`

Este método permite calcular la tasa de interés correspondiente a un dado factor de capitalización, utilizando la función de capitalización y la fracción de año de la tasa. El intervalor de tiempo de la tasa se puede especificar con un par de fechas o con un número de días.

In [34]:
factor = 1.0025
dias = 31
for tasa in tasas:
    aux = f"{tasa}"[-10:]
    print(f"Tasa: {aux}")
    print(f"get_rate_from_wf(factor, fecha1, fecha3): {tasa.get_rate_from_wf(factor, fecha1, fecha3):.4%}")
    print(f"get_rate_from_wf(factor, dias): {tasa.get_rate_from_wf(factor, dias):.4%}\n")


Tasa: Act360 Lin
get_rate_from_wf(factor, fecha1, fecha3): 3.0000%
get_rate_from_wf(factor, dias): 2.9032%

Tasa: Act365 Com
get_rate_from_wf(factor, fecha1, fecha3): 3.0845%
get_rate_from_wf(factor, dias): 2.9835%

Tasa: Act365 Exp
get_rate_from_wf(factor, fecha1, fecha3): 3.0379%
get_rate_from_wf(factor, dias): 2.9399%



## Tenor

Es una clase que representa el concepto de plazo estructurado o tenor (1D, 1M, 1Y ...).

### Ejemplos

In [185]:
tenors = [
    _1d:=qcf.Tenor("1d"),
    _1m:=qcf.Tenor("1M"),
    _1y:=qcf.Tenor("1y"),
    _1d_1m_1y:=qcf.Tenor("1D1M1Y"),

    # Notar que, en este caso, el constructor es capaz de eliminar
    # los espacios y la substr nyse
    _2y_3m:=qcf.Tenor("2y nyse 3m"),  
]

### Métodos `get_string`, `get_days`, `get_months` y `get_years`

In [187]:
for tenor in tenors:
    print(f"string: {tenor.get_string()}")
    print(f"dias: {tenor.get_days()}")
    print(f"meses: {tenor.get_months()}")
    print(f"años: {tenor.get_years()}\n")

string: 1D
dias: 1
meses: 0
años: 0

string: 1M
dias: 0
meses: 1
años: 0

string: 1Y
dias: 0
meses: 0
años: 1

string: 1Y1M1D
dias: 1
meses: 1
años: 1

string: 2Y3M
dias: 0
meses: 3
años: 2



### Método `set_tenor`

In [188]:
for i, tenor in enumerate(tenors):
    tenor.set_tenor(f"{i}d{i}m{i}y")
    print(f"string: {tenor.get_string()}\n")

string: 0D

string: 1Y1M1D

string: 2Y2M2D

string: 3Y3M3D

string: 4Y4M4D



## FX Rate

Es una clase que representa el concepto de tipo de cambio entre dos monedas. Para dar de alta un FXRate se requiere:

- QCCurrency: la moneda fuerte del par.
- QCCurrency: la moneda débl del par.

### Ejemplo: USDCLP

In [189]:
usdclp = qcf.FXRate(usd, clp)

Utilizando el método `get_code` se puede obtener el código del par según la convención usual.

In [190]:
print(f"Código: {usdclp.get_code()}")

Código: USDCLP


## FX Rate Index

Esta clase representa un índice de tipo de cambio, por ejemplo, el dólar observado que publica el Banco Central de Chile.

Para dar de alta un FXRateIndex se requiere:

- `FXRate`: el FXRate correspoondiente.
- `str`: nombre del índice
- `Tenor`: la regla de fixing, es 1D como el USD Observado o es 0D como un ínidce de cierre de día.
- `Tenor`: la regla para la valuta. Es 1D como el USDCLP o 2D como el EURUSD.
- `BusinessCalendar`: el calendario adecuado para aplicar las reglas de fixing y valuta.

### Ejemplo

In [198]:
_1d.set_tenor("1d")
usdclp_obs = qcf.FXRateIndex(usdclp, "USDOBS", _1d, _1d, scl)

### Métodos `fixing_date` y `value_date`

El método `fixing_date` retorna la fecha de fixing del índice dada la fecha de publicación. Por su parte, `value_date` retorna la fecha de la valuta dada la fecha de publicación.

In [200]:
print(f"Fecha de publicación: {fecha1.week_day()}, {fecha1}")
print(f"Fecha de fixing: {usdclp_obs.fixing_date(fecha1)}")
print(f"Fecha de valuta: {usdclp_obs.value_date(fecha1)}")

Fecha de publicación: WeekDay.THU, 2024-10-17
Fecha de fixing: 2024-10-16
Fecha de valuta: 2024-10-17


Notar que la fecha de fixing se calcula aplicando la regla de fixing a la fecha de publicación, mientras que la fecha de valuta se calcula aplicando la regla de valuta a la fecha de fixing.

### Método `convert`

El método `convert` permite pasar rápidamente de una moneda a la otra (de las que forman el par del índice) usando yun valor para el índice.

In [202]:
monto_usd = 1_000_000
monto_clp = 900_000_000
valor_usdclp_obs = 900.00

result = usdclp_obs.convert(monto_usd, qcf.QCUSD(), valor_usdclp_obs)
print(f"Monto en CLP es: {result:,.0f}")

result = usdclp_obs.convert(monto_clp, qcf.QCCLP(), valor_usdclp_obs)
print(f"Monto en USD es: {result:,.0f}")

Monto en CLP es: 900,000,000
Monto en USD es: 1,000,000


Esta función es cómoda porque evita tener que controlar en el propio código si la divisa del monto a convertir es la fuerte o la débil del par.

## QCCurrencyConverter

Este es un objeto que permite realizar conversiones de una moneda a otra con un poco más de generalidad que en el caso anterior.

In [42]:
ccy_converter = qcf.QCCurrencyConverter()

In [43]:
print(f'Monto en CLP: {ccy_converter.convert(1_000, usd, 800, usdclp_obs):,.0f}')

Monto en CLP: 800,000


In [44]:
print(f'Monto en USD: {ccy_converter.convert(800_000, clp, 800, usdclp_obs):,.0f}')

Monto en USD: 1,000


#### Enum para Monedas

In [45]:
qcf.QCCurrencyEnum.AUD

<QCCurrencyEnum.AUD: 0>

In [46]:
qcf.QCCurrencyEnum.USD

<QCCurrencyEnum.USD: 18>

#### Enum para FXRates

In [47]:
qcf.QCFxRateEnum.USDCLP

<QCFxRateEnum.USDCLP: 14>

In [48]:
qcf.QCFxRateEnum.EURUSD

<QCFxRateEnum.EURUSD: 30>

Ambos `enum` se pueden utilizar con el método `convert`:

In [49]:
print(f'Monto en USD: {ccy_converter.convert(800_000, qcf.QCCurrencyEnum.CLP, 800, qcf.QCFxRateEnum.USDCLP):,.0f}')

Monto en USD: 1,000


In [50]:
print(f'Monto en USD: {ccy_converter.convert(1_000, qcf.QCCurrencyEnum.USD, 800, qcf.QCFxRateEnum.USDCLP):,.0f}')

Monto en USD: 800,000


## Time Series

In [51]:
ts = qcf.time_series()
ts[fecha1] = 10.0

In [52]:
type(ts)

qcfinancial.time_series

In [53]:
for k, v in ts.items():
    print(k, v)

2024-10-17 10.0


## Cashflows

### Simple Cashflow
Un objeto de tipo `SimpleCashflow` representa un flujo de caja cierto en una fecha y moneda.

In [54]:
# Como se inicializa un objeto SimpleCashflow.
simple_cashflow = qcf.SimpleCashflow(
    fecha1, 100, clp  # fecha del flujo  # monto
)  # moneda

In [55]:
simple_cashflow

In [56]:
# Métodos amount, ccy y date
print(simple_cashflow.amount())
print(simple_cashflow.ccy().get_iso_code())
print(simple_cashflow.date())

100.0
CLP
2024-10-17


 Primer ejemplo de la función `show`. Envuelve de forma cómoda todo el flujo en una tupla. La función show está sobregargada y admite muchos tipos de flujo.

In [57]:
qcf.show(simple_cashflow)

('2024-10-17', 100.0, 'CLP')

### Simple Multicurrency Cashflow
Un objeto de tipo `SimpleMultiCurrencyCashflow` representa un flujo de caja cierto en una fecha y moneda, que, sin embargo, se liquidará en una segunda moneda utilizando el valor de un índice de tipo de cambio. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha final
- `float`: nominal
- `QCCurrency`: la moneda del nominal
- `QCDate`: la fecha de publicación del `FXRateIndex` que se usará en la conversión
- `QCCurrency`: la moneda final del flujo
- `FXRateIndex`: el índice de tipo de cambio a utilizar
- `float`: el valor del índice de tipo de cambio

In [58]:
# Ejemplo.
simple_mccy_cashflow = qcf.SimpleMultiCurrencyCashflow(
    fecha3, 700.00, clp, fecha3, usd, usdclp_obs, 700.00
)
print("fecha flujo:", simple_mccy_cashflow.date())
print("nominal:", simple_mccy_cashflow.nominal())
print("moneda nominal:", simple_mccy_cashflow.ccy().get_iso_code())
print("flujo:", simple_mccy_cashflow.amount())
print("moneda flujo:", simple_mccy_cashflow.settlement_ccy().get_iso_code())

fecha flujo: 2024-11-16
nominal: 700.0
moneda nominal: CLP
flujo: 1.0
moneda flujo: USD


In [59]:
# El valor del índice se puede alterar (existe un setter).
simple_mccy_cashflow.set_fx_rate_index_value(800.00)
print("flujo:", simple_mccy_cashflow.amount())

flujo: 0.875


In [60]:
# Ejemplo 2. Las monedas están invertidas respecto al caso anterior. El valor del índice se usa de la forma correcta.
simple_mccy_cashflow = qcf.SimpleMultiCurrencyCashflow(
    fecha3, 1, usd, fecha3, clp, usdclp_obs, 700.00
)
print("fecha flujo:", simple_mccy_cashflow.date())
print("nominal:", simple_mccy_cashflow.nominal())
print("moneda nominal:", simple_mccy_cashflow.ccy().get_iso_code())
print("flujo:", simple_mccy_cashflow.amount())
print("moneda flujo:", simple_mccy_cashflow.settlement_ccy().get_iso_code())

fecha flujo: 2024-11-16
nominal: 1.0
moneda nominal: USD
flujo: 700.0
moneda flujo: CLP


In [61]:
# Ejemplo 3. Hay inconsistencia entre las monedas y el tipo de cambio del índice.
eur = qcf.QCEUR()
try:
    simple_mccy_cashflow = qcf.SimpleMultiCurrencyCashflow(
        fecha3, 1, usd, fecha3, eur, usdclp_obs, 700.00
    )
except ValueError as e:
    print("Error:", e)

Error: Fx Rate Index provided is not compatible with nominal and settlement currency. 


In [62]:
# Función show
qcf.show(simple_mccy_cashflow)

('2024-11-16', 1.0, 'USD', '2024-11-16', 'CLP', 'USDOBS', 700.0, 700.0)

In [63]:
simple_mccy_cashflow.get_type()

'SimpleMultiCurrencyCashflow'

### Fixed Rate Cashflow
Un objeto de tipo `FixedRateCashflow` representa un flujo de caja calculado a partir de la aplicación de una tasa prefijada, entre dos fechas prefijadas a un nominal prefijado. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `QCInterestRate`: la tasa de interés a aplicar (su valor y convenciones)
- `QCCurrency`: moneda del nominal y del flujo de caja

In [64]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2018)
fecha_final = qcf.QCDate(20, 9, 2019)
fecha_pago = qcf.QCDate(23, 9, 2019)
tasa = qcf.QCInterestRate(0.1, act360, lin_wf)
fixed_rate_cashflow = qcf.FixedRateCashflow(
    fecha_inicio, 
    fecha_final, 
    fecha_pago, 
    1_000_000_000.0, 
    100_000_000.0, 
    True, 
    tasa, 
    clp
)

In [65]:
# Getters
print("Fecha Inicio:", fixed_rate_cashflow.get_start_date())
print("Fecha Final:", fixed_rate_cashflow.get_end_date())
print("Fecha Pago:", fixed_rate_cashflow.get_settlement_date())
print("Moneda:", fixed_rate_cashflow.ccy().get_iso_code())
print(f"Nominal: {fixed_rate_cashflow.get_nominal():,.0f}")
print(f"Amortización: {fixed_rate_cashflow.get_amortization():,.0f}")

Fecha Inicio: 2018-09-20
Fecha Final: 2019-09-20
Fecha Pago: 2019-09-23
Moneda: CLP
Nominal: 1,000,000,000
Amortización: 100,000,000


In [66]:
# Setters
# TODO: set_rate
nuevo_nominal = 2_000_000_000.0
fixed_rate_cashflow.set_nominal(nuevo_nominal)
print(f"Nuevo nominal: {fixed_rate_cashflow.get_nominal():,.0f}")

nueva_amortizacion = 200_000_000.0
fixed_rate_cashflow.set_amortization(nueva_amortizacion)
print(f"Nueva amortización: {fixed_rate_cashflow.get_amortization():,.0f}")

Nuevo nominal: 2,000,000,000
Nueva amortización: 200,000,000


In [67]:
# Cálculos
print(f"Flujo Total: {fixed_rate_cashflow.amount():,.0f}")
print(f"Check: {fixed_rate_cashflow.get_nominal() * 0.1 * 365.0 / 360 + fixed_rate_cashflow.get_amortization():,.0f}")
print()
fecha_intermedia = qcf.QCDate(2, 1, 2019)
print(f"Interés Devengado: {fixed_rate_cashflow.accrued_interest(fecha_intermedia):,.0f}")
print(f"Check: {fixed_rate_cashflow.get_nominal() * 0.1 * fecha_inicio.day_diff(fecha_intermedia) / 360.0:,.0f}")

Flujo Total: 402,777,778
Check: 402,777,778

Interés Devengado: 57,777,778
Check: 57,777,778


In [68]:
# Función show
print(qcf.show(fixed_rate_cashflow))

('2018-09-20', '2019-09-20', '2019-09-23', 2000000000.0, 200000000.0, 202777777.77777794, True, 402777777.7777779, 'CLP', 0.1, 'LinAct360')


In [69]:
print(f"Interés total: al {fixed_rate_cashflow.accrued_interest(fixed_rate_cashflow.get_end_date()):,.0f}")

Interés total: al 202,777,778


In [70]:
fixed_rate_cashflow.get_type()

'FixedRateCashflow'

### Fixed Rate Multi Currency Cashflow
Un objeto de tipo `FixedRateMultiCurrencyCashflow` representa un flujo de caja a tasa fija (`FixedRateCashflow`) que se liquidará en una moneda distinta de la moneda del nominal utilizando el valor a una cierta fecha de un índice de tipo de cambio prefijado. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `QCInterestRate`: la tasa de interés a aplicar (su valor y convenciones)
- `QCCurrency`: moneda del nominal
- `QCDate`: fecha de publicación del índice de tipo de cambio
- `QCCurrency`: moneda en la que se liquida el flujo
- `FXRateIndex`: índice de tipo de cambio a utilizar
- `float`: valor del índice de tipo de cambio

In [71]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2018)
fecha_final = qcf.QCDate(20, 9, 2019)
fecha_pago = qcf.QCDate(23, 9, 2019)
fecha_publicacion = qcf.QCDate(23, 9, 2019)
usd = qcf.QCUSD()
indice = usdclp_obs
valor_indice = 20.0
nominal = 1_000.0
amort = 1_000.0
fixed_rate_mccy_cashflow = qcf.FixedRateMultiCurrencyCashflow(
    fecha_inicio,
    fecha_final,
    fecha_pago,
    nominal,
    amort,
    False,
    tasa,
    usd,
    fecha_publicacion,
    clp,
    indice,
    valor_indice,
)
print(fixed_rate_mccy_cashflow)

**TODO: get_rate.** Este getter no debe ser un getter tradicional, ya que no es necesario que retorne una referencia a todo el objeto ``QCInterestRate``, basta con el valor y la descripción de yf y wf asociado (mejor llamarlo get_rate_info).

In [72]:
# Getters
print("Fecha Inicio:", fixed_rate_mccy_cashflow.get_start_date())
print("Fecha Final:", fixed_rate_mccy_cashflow.get_end_date())
print("Fecha Pago:", fixed_rate_mccy_cashflow.get_settlement_date())
print("Fecha Fijación Índice FX:", fixed_rate_mccy_cashflow.get_fx_fixing_date())
print("Moneda del Nominal:", fixed_rate_mccy_cashflow.ccy())
print(f"Nominal: {fixed_rate_mccy_cashflow.get_nominal():,.0f}")
print(f"Amortización: {fixed_rate_mccy_cashflow.get_amortization():,.0f}")
print("Moneda de Liquidación:", fixed_rate_mccy_cashflow.settlement_currency())

Fecha Inicio: 2018-09-20
Fecha Final: 2019-09-20
Fecha Pago: 2019-09-23
Fecha Fijación Índice FX: 2019-09-23
Moneda del Nominal: USD
Nominal: 1,000
Amortización: 1,000
Moneda de Liquidación: CLP


**TODO: set_rate_value.** Debe establecer el valor de la tasa de interés.

In [73]:
# Setters
nuevo_nominal = 100.0
fixed_rate_mccy_cashflow.set_nominal(nuevo_nominal)
print(f"Nuevo nominal: {fixed_rate_mccy_cashflow.get_nominal():,.1f}")

nueva_amortizacion = 10.0
fixed_rate_mccy_cashflow.set_amortization(nueva_amortizacion)
print(f"Nueva amortización: {fixed_rate_mccy_cashflow.get_amortization():,.1f}")

Nuevo nominal: 100.0
Nueva amortización: 10.0


In [74]:
# Cálculos
fixed_rate_mccy_cashflow.set_nominal(nominal)
fixed_rate_mccy_cashflow.set_amortization(amort)
print(f"Flujo Total: {fixed_rate_mccy_cashflow.amount():,.2f}")
print(f"Check: {(fixed_rate_mccy_cashflow.get_nominal() * 0.1 * 365.0 / 360) * valor_indice:,.2f}")
print()
fecha_intermedia = qcf.QCDate(2, 1, 2019)
print(f"Interés Devengado: {fixed_rate_mccy_cashflow.accrued_interest(fecha_intermedia):,.02f}")
print(f"Check: {fixed_rate_mccy_cashflow.get_nominal() * 0.1 * fecha_inicio.day_diff(fecha_intermedia) / 360.0:,.02f}")

Flujo Total: 101.39
Check: 2,027.78

Interés Devengado: 28.89
Check: 28.89


In [75]:
ts = qcf.time_series()

In [76]:
ts[fecha_inicio] = 10

In [77]:
ts[fecha_inicio]

10.0

In [78]:
ts[fecha_intermedia] = 15

In [79]:
print(fixed_rate_mccy_cashflow.accrued_interest(fecha_intermedia, fecha_inicio, ts))

288.88888888888965


In [80]:
print(fixed_rate_mccy_cashflow.accrued_interest(fecha_intermedia, fecha_intermedia, ts))

433.33333333333445


In [81]:
# Función show
print(qcf.show(fixed_rate_mccy_cashflow))

('2018-09-20', '2019-09-20', '2019-09-23', 1000.0, 1000.0, 101.38888888888897, False, 101.38888888888897, 'USD', 0.1, 'LinAct360', '2019-09-23', 'CLP', 'USDOBS', 20.0, 20000.0, 2027.7777777777794)


In [82]:
fixed_rate_mccy_cashflow.get_type()

'FixedRateMultiCurrencyCashflow'

### Fixed Rate Cashflow 2
Un objeto de tipo `FixedRateCashflow2` representa un flujo de caja calculado a partir de la aplicación de una tasa prefijada, entre dos fechas prefijadas a un nominal prefijado. Este tipo de cashflow puede ser *quantizado*, es decir, se puede cambiar su moneda de pago componiéndolo con un objeto adicional. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `QCInterestRate`: la tasa de interés a aplicar (su valor y convenciones)
- `QCCurrency`: moneda del nominal y del flujo de caja

In [83]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2018)
fecha_final = qcf.QCDate(20, 9, 2019)
fecha_pago = qcf.QCDate(23, 9, 2019)
tasa = qcf.QCInterestRate(0.15, act360, lin_wf)
fixed_rate_cashflow_2 = qcf.FixedRateCashflow2(
    fecha_inicio, 
    fecha_final, 
    fecha_pago, 
    1_000_000_000.0,
    100_000_000.0, 
    True, 
    tasa, 
    clp
)

In [84]:
print(f"Accrued Fixing: {fixed_rate_cashflow_2.accrued_fixing(fecha_inicio):.2%}")

Accrued Fixing: 15.00%


In [85]:
print(f"amount: {fixed_rate_cashflow_2.amount():,.0f}")
print("currency:", fixed_rate_cashflow_2.ccy().get_iso_code())
print("date:", fixed_rate_cashflow_2.date())
print("start date:", fixed_rate_cashflow_2.get_start_date())
print("end date:", fixed_rate_cashflow_2.get_end_date())
print("settlement date:", fixed_rate_cashflow_2.get_settlement_date())
print("fixing dates:")
for f in fixed_rate_cashflow_2.get_fixing_dates():
    print("\t", f)
print(f"nominal: {fixed_rate_cashflow_2.get_nominal():,.0f}")
print(f"amortization: {fixed_rate_cashflow_2.get_amortization():,.0f}")
print(f"interest (1st overload): {fixed_rate_cashflow_2.interest():,.0f}")

ts = qcf.time_series()
print(f"interest (2nd overload): {fixed_rate_cashflow_2.interest(ts):,.0f}")
print(f"fixing (1st overload): {fixed_rate_cashflow_2.fixing():.2%}")
print(f"fixing (2nd overload): {fixed_rate_cashflow_2.fixing(ts):.2%}")
fecha_intermedia = qcf.QCDate(20, 3, 2019)
print(f"accrued interest (1st overload): {fixed_rate_cashflow_2.accrued_interest(fecha_intermedia):,.0f}")
print(f"accrued interest (2nd overload): {fixed_rate_cashflow_2.accrued_interest(fecha_intermedia, ts):,.0f}")
print(f"accrued fixing (1st overload): {fixed_rate_cashflow_2.accrued_fixing(fecha_intermedia):.2%}")
print(f"accrued fixing (2nd overload): {fixed_rate_cashflow_2.accrued_fixing(fecha_intermedia, ts):,.2%}")

amount: 252,083,333
currency: CLP
date: 2019-09-23
start date: 2018-09-20
end date: 2019-09-20
settlement date: 2019-09-23
fixing dates:
	 2018-09-20
nominal: 1,000,000,000
amortization: 100,000,000
interest (1st overload): 152,083,333
interest (2nd overload): 152,083,333
fixing (1st overload): 15.00%
fixing (2nd overload): 15.00%
accrued interest (1st overload): 75,416,667
accrued interest (2nd overload): 75,416,667
accrued fixing (1st overload): 15.00%
accrued fixing (2nd overload): 15.00%


In [86]:
fixed_rate_cashflow_2.get_type()

'FixedRateCashflow2'

In [87]:
qcf.show(fixed_rate_cashflow_2)

('2018-09-20',
 '2019-09-20',
 '2019-09-23',
 1000000000.0,
 100000000.0,
 152083333.33333334,
 True,
 252083333.33333334,
 'CLP',
 0.15,
 'LinAct360')

### Ibor Cashflow
Un objeto de tipo `IborCashflow` representa un flujo de caja calculado a partir de la aplicación de una tasa flotante fijada en una cierta fecha (Libor, Euribor, TAB, ...) , entre dos fechas prefijadas a un nominal prefijado. Para dar de alta uno de estos objetos se requiere:

- `InterestRateIndex`: el índice de tasa de interés prefijado
- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de fijación del índice de tasa de interés 
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `QCCurrency`: moneda del nominal y del flujo de caja
- `float`: spread aditivo a aplicar a la fijación del índice
- `float`: spread multiplicativo o gearing a aplicar a la fijación del índice

Para dar de alta un `InterestRateIndex` se requiere:

- `str`: código del índice
- `QCInterestRate`: un objeto tasa de interés que contenga las convenciones del índice
- `Tenor`: el lag de inicio del índice respecto a la fecha de fixing (por ejemplo 2d para Libor USD)
- `Tenor`: el tenor del índice (3M por ejemplo para Libor USD 3M)
- `QCBusinessCalendar`: el calendario de fixing
- `QCBusinessCalendar`: el calendario de pago
- `QCCurrency`: la moneda a la que corresponde el índice (por ejemplo EUR para EURIBOR 3M)


**TODO: agregar end_date_adjustment al objeto (FOLLOW o MOD FOLLOW).** Esto debiera tener un correspondiente cambio en la BBDD de Front Desk.

In [88]:
# Se define el índice
codigo = "LIBORUSD3M"
lin_act360 = qcf.QCInterestRate(0.0, act360, lin_wf)
fixing_lag = qcf.Tenor("2d")
tenor = qcf.Tenor("3m")
fixing_calendar = scl  # No es el calendario correcto, pero sirve para el ejemplo
settlement_calendar = scl  # Ídem arriba
libor_usd_3m = qcf.InterestRateIndex(
    codigo, 
    lin_act360, 
    fixing_lag, 
    tenor, 
    fixing_calendar, 
    settlement_calendar, 
    usd
)

# Getters
print("Tenor:", libor_usd_3m.get_tenor())
print("Tasa:", libor_usd_3m.get_rate())
print()

# Para construir un fixing en particular
libor_usd_3m.set_rate_value(0.01)
print("Fixing Tasa:", libor_usd_3m.get_rate())
fecha_fixing = qcf.QCDate(20, 9, 2018)
print("Fecha Inicio:", libor_usd_3m.get_start_date(fecha_fixing))
print("Fecha Final:", libor_usd_3m.get_end_date(fecha_fixing))

Tenor: 3M
Tasa: 0.000000 Act360 Lin

Fixing Tasa: 0.010000 Act360 Lin
Fecha Inicio: 2018-09-24
Fecha Final: 2018-12-24


In [89]:
libor_usd_3m.get_code()

'LIBORUSD3M'

In [90]:
libor_usd_3m.set_rate_value(0.1)

In [91]:
print(libor_usd_3m.get_rate())

0.100000 Act360 Lin


In [92]:
libor_usd_3m.get_rate()

Con esto, veamos un ejemplo de construcción y uso de un `IborCashflow`.

**TODO:** Se debe crear un mecanismo de WARNING para las eventuales inconsistencias entre las fechas de inicio y fin del ``InterestRateIndex`` y las fechas de inicio y fin del ``IborCashflow``.

In [93]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2018)
fecha_final = qcf.QCDate(20, 9, 2019)
fecha_pago = qcf.QCDate(23, 9, 2019)
fecha_fixing = qcf.QCDate(20, 9, 2018)
nominal = 1_000_000.0
amort = 100_000.0
spread = 0.0
gearing = 1.0
ibor_cashflow = qcf.IborCashflow(
    libor_usd_3m,
    fecha_inicio,
    fecha_final,
    fecha_fixing,
    fecha_pago,
    nominal,
    amort,
    True,
    usd,
    spread,
    gearing,
)

In [94]:
# Getters
print("Fecha Fixing:\t", ibor_cashflow.get_fixing_date())
print("Fecha Inicio:\t", ibor_cashflow.get_start_date())
print("Fecha Final:\t", ibor_cashflow.get_end_date())
print("Fecha Pago:\t", ibor_cashflow.get_settlement_date())
print(f"Nominal:\t{ibor_cashflow.get_nominal():,.0f}")
print(f"Amortización:\t{ibor_cashflow.get_amortization():,.0f}")
print("Moneda:\t\t", ibor_cashflow.ccy())
print(f"Valor Tasa:\t{ibor_cashflow.get_interest_rate_value():.2%}")

Fecha Fixing:	 2018-09-20
Fecha Inicio:	 2018-09-20
Fecha Final:	 2019-09-20
Fecha Pago:	 2019-09-23
Nominal:	1,000,000
Amortización:	100,000
Moneda:		 USD
Valor Tasa:	10.00%


In [95]:
# Setters
nuevo_nominal = 2_000_000.0
ibor_cashflow.set_nominal(nuevo_nominal)
print(f"Nominal:\t{ibor_cashflow.get_nominal():,.0f}")

nueva_amortizacion = 200_000.0
ibor_cashflow.set_amortization(nueva_amortizacion)
print(f"Amortización:\t{ibor_cashflow.get_amortization():,.0f}")

nuevo_valor_tasa = 0.02
ibor_cashflow.set_interest_rate_value(nuevo_valor_tasa)
print(f"Valor Tasa: {ibor_cashflow.get_interest_rate_value():.2%}")

Nominal:	2,000,000
Amortización:	200,000
Valor Tasa: 2.00%


In [96]:
# Cálculos
print(f"Flujo: {ibor_cashflow.amount():,.0f}")

fecha_devengo = qcf.QCDate(20, 7, 2019)
print(f"Interés Devengado al {fecha_devengo}: {ibor_cashflow.accrued_interest(fecha_devengo):,.0f}")
tasa = ibor_cashflow.get_interest_rate_value()

check = tasa * fecha_inicio.day_diff(fecha_devengo) / 360.0 * ibor_cashflow.get_nominal()
print(f"Check: {check:,.0f}")

Flujo: 240,556
Interés Devengado al 2019-07-20: 33,667
Check: 33,667


In [97]:
# Función show
print(qcf.show(ibor_cashflow))

('2018-09-20', '2019-09-20', '2018-09-20', '2019-09-23', 2000000.0, 200000.0, 40555.55555555568, True, 240555.55555555568, 'USD', 'LIBORUSD3M', 0.02, 0.0, 1.0, 'LinAct360')


### Ibor Cashflow 2
Un objeto de tipo `IborCashflow2` representa un flujo de caja calculado a partir de la aplicación de una tasa flotante fijada en una cierta fecha (Libor, Euribor, TAB, ...) , entre dos fechas prefijadas a un nominal prefijado. Para dar de alta uno de estos objetos se requiere:

- `InterestRateIndex`: el índice de tasa de interés prefijado
- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de fijación del índice de tasa de interés 
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `QCCurrency`: moneda del nominal y del flujo de caja
- `float`: spread aditivo a aplicar a la fijación del índice
- `float`: spread multiplicativo o gearing a aplicar a la fijación del índice

A diferencia de un `IborCashflow`, un `IborCashflow2` puede quantizarse.

In [98]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2018)
fecha_final = qcf.QCDate(20, 9, 2019)
fecha_pago = qcf.QCDate(23, 9, 2019)
fecha_fixing = qcf.QCDate(20, 9, 2018)
nominal = 1000000.0
amort = 100000.0
spread = 0.0
gearing = 1.0
ibor_cashflow_2 = qcf.IborCashflow2(
    libor_usd_3m,
    fecha_inicio,
    fecha_final,
    fecha_fixing,
    fecha_pago,
    nominal,
    amort,
    True,
    usd,
    spread,
    gearing,
)

In [99]:
# Getters
print("Fecha Fixing:\t", ibor_cashflow_2.get_fixing_dates())
print("Fecha Inicio:\t", ibor_cashflow_2.get_start_date())
print("Fecha Final:\t", ibor_cashflow_2.get_end_date())
print("Fecha Pago:\t", ibor_cashflow_2.get_settlement_date())
print(f"Nominal:\t{ibor_cashflow_2.get_nominal():,.0f}")
print(f"Amortización:\t{ibor_cashflow_2.get_amortization():,.0f}")
print("Moneda:\t\t", ibor_cashflow_2.ccy())
print(f"Valor Tasa:\t{ibor_cashflow_2.fixing():.2%}")

Fecha Fixing:	 DateList[20-9-2018]
Fecha Inicio:	 2018-09-20
Fecha Final:	 2019-09-20
Fecha Pago:	 2019-09-23
Nominal:	1,000,000
Amortización:	100,000
Moneda:		 USD
Valor Tasa:	2.00%


In [100]:
# Función show
qcf.show(ibor_cashflow_2)

('2018-09-20',
 '2019-09-20',
 '2018-09-20',
 '2019-09-23',
 1000000.0,
 100000.0,
 20277.77777777784,
 True,
 120277.77777777784,
 'USD',
 'LIBORUSD3M',
 0.02,
 0.0,
 1.0,
 'LinAct360')

### Ibor Multi Currency Cashflow
Un objeto de tipo `IborMultiCurrencyCashflow` representa un flujo de caja a tasa variable (`IborCashflow`) que se liquidará en una moneda distinta de la moneda del nominal utilizando el valor a una cierta fecha de un índice de tipo de cambio prefijado. Para dar de alta uno de estos objetos se requiere:

- `InterestRateIndex`: el índice de tasa de interés prefijado
- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de fijación del índice de tasa de interés 
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `QCCurrency`: moneda del nominal y del flujo de caja
- `float`: spread aditivo a aplicar a la fijación del índice
- `float`: spread multiplicativo o gearing a aplicar a la fijación del índice
- `QCDate`: fecha de publicación del índice de tipo de cambio
- `QCCurrency`: moneda en la que se liquida el flujo
- `FXRateIndex`: índice de tipo de cambio a utilizar
- `float`: valor del índice de tipo de cambio


In [101]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2019)
fecha_final = qcf.QCDate(20, 12, 2019)
fecha_pago = qcf.QCDate(20, 12, 2019)
fecha_fixing = qcf.QCDate(20, 9, 2019)
nominal = 100.0
amort = 100.0
spread = 0.02
gearing = 1.0
valor_indice = 10.0
fecha_publicacion = qcf.QCDate(20, 9, 2019)
libor_usd_3m.set_rate_value(0.01)
ibor_mccy_cashflow = qcf.IborMultiCurrencyCashflow(
    libor_usd_3m,
    fecha_inicio,
    fecha_final,
    fecha_fixing,
    fecha_pago,
    nominal,
    amort,
    True,
    usd,
    spread,
    gearing,
    fecha_publicacion,
    clp,
    indice,
    valor_indice,
)
print(ibor_mccy_cashflow)

In [102]:
ibor_mccy_cashflow.set_interest_rate_value(0.01)

In [103]:
print(qcf.show(ibor_mccy_cashflow))

('2019-09-20', '2019-12-20', '2019-09-20', '2019-12-20', 100.0, 100.0, 0.7583333333333275, True, 1007.5833333333333, 'USD', 'LIBORUSD3M', 0.02, 1.0, 0.01, 'LinAct360', '2019-09-20', 'CLP', 'USDOBS', 10.0, 1000.0, 7.583333333333275)


In [104]:
fecha_intermedia = qcf.QCDate(20, 10, 2019)
print(ibor_mccy_cashflow.accrued_interest(fecha_intermedia))
print(
    (0.01 + spread)
    * fecha_inicio.day_diff(fecha_intermedia)
    / 360.0
    * ibor_mccy_cashflow.get_nominal()
)

0.24999999999999467
0.24999999999999997


In [105]:
ts[fecha_inicio] = 1
ts[fecha_intermedia] = 3
for k in ts:
    print(k)
print(fecha_intermedia.description(False))
print(ts[fecha_intermedia])

2019-09-20
2019-10-20
2019-10-20
3.0


In [106]:
print(ibor_mccy_cashflow.accrued_interest(fecha_intermedia, fecha_intermedia, ts))

0.749999999999984


In [107]:
ibor_mccy_cashflow.get_type()

'IborMultiCurrencyCashflow'

In [108]:
qcf.show(ibor_mccy_cashflow)

('2019-09-20',
 '2019-12-20',
 '2019-09-20',
 '2019-12-20',
 100.0,
 100.0,
 0.7583333333333275,
 True,
 1007.5833333333333,
 'USD',
 'LIBORUSD3M',
 0.02,
 1.0,
 0.01,
 'LinAct360',
 '2019-09-20',
 'CLP',
 'USDOBS',
 10.0,
 1000.0,
 7.583333333333275)

### Icp Clp Cashflow
Un objeto de tipo `IcpClpCashflow` representa un flujo de caja calculado como un cupón de la pata flotante de un swap ICP (cámara promedio) de Chile. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `float`: spread aditivo a aplicar a la fijación de la TNA
- `float`: spread multiplicativo o gearing a aplicar a la fijación de la TNA
- `float`: el valor del ICP a fecha de inicio (u otro valor arbitrario si el valor es desconocido)
- `float`: el valor del ICP a fecha final (u otro valor arbitrario si el valor es desconocido)

Recordar que TNA significa **Tasa Nominal Anual** y se determina utilizando los valores del índice ICP en la fecha de inicio y fecha final del `IcpClpCashflow`.

In [109]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2018)
fecha_final = qcf.QCDate(20, 9, 2019)
fecha_pago = qcf.QCDate(23, 9, 2019)
nominal = 1_000_000_000.0
amort = 100_000_000.0
spread = 0.0
gearing = 1.0
icp_clp_cashflow = qcf.IcpClpCashflow(
    fecha_inicio,
    fecha_final,
    fecha_pago,
    nominal,
    amort,
    True,
    spread,
    gearing,
    10_000.0,
    10_250.0,
)

In [110]:
# Getters
print("Fecha Inicio:", icp_clp_cashflow.get_start_date())
print("Fecha Final:", icp_clp_cashflow.get_end_date())
print(f"ICP Fecha Inicio: {icp_clp_cashflow.get_start_date_icp():,.2f}")
print(f"ICP Fecha Final: {icp_clp_cashflow.get_end_date_icp():,.2f}")
print(f"Valor TNA Todo el Período: {icp_clp_cashflow.get_rate_value():.2%}")
check = round((10250.0 / 10000 - 1) * 360.0 / fecha_inicio.day_diff(fecha_final), 4)
print(f"Check: {check:.2%}")
print(f"Nominal: {icp_clp_cashflow.get_nominal():,.0f}")
print(f"Amortización: {icp_clp_cashflow.get_amortization():,.0f}")
print("Tipo de Tasa:", icp_clp_cashflow.get_type_of_rate())
print("Moneda:", icp_clp_cashflow.ccy())

Fecha Inicio: 2018-09-20
Fecha Final: 2019-09-20
ICP Fecha Inicio: 10,000.00
ICP Fecha Final: 10,250.00
Valor TNA Todo el Período: 2.47%
Check: 2.47%
Nominal: 1,000,000,000
Amortización: 100,000,000
Tipo de Tasa: LinAct360
Moneda: CLP


In [111]:
# Setters
decimales_para_tna = 6
icp_clp_cashflow.set_tna_decimal_places(decimales_para_tna)
print(f"Nueva TNA: {icp_clp_cashflow.get_rate_value():.4%}")

nuevo_nominal = 100
icp_clp_cashflow.set_nominal(nuevo_nominal)
print("Nuevo Nominal:", icp_clp_cashflow.get_nominal())

nueva_amortizacion = 10
icp_clp_cashflow.set_amortization(nueva_amortizacion)
print("Nueva Amortización:", icp_clp_cashflow.get_amortization())

nuevo_icp_inicio = 20_000
icp_clp_cashflow.set_start_date_icp(nuevo_icp_inicio)
print(f"Nuevo ICP Inicio: {icp_clp_cashflow.get_start_date_icp():,.2f}")

nuevo_icp_final = 20_600
icp_clp_cashflow.set_end_date_icp(nuevo_icp_final)
print(f"Nuevo ICP Final: {icp_clp_cashflow.get_end_date_icp():,.2f}")
print(f"Check TNA Final: {icp_clp_cashflow.get_rate_value():.4%}")

Nueva TNA: 2.4658%
Nuevo Nominal: 100.0
Nueva Amortización: 10.0
Nuevo ICP Inicio: 20,000.00
Nuevo ICP Final: 20,600.00
Check TNA Final: 2.9589%


In [112]:
# Cálculos
decimales_para_tna = 4  # Se vuelve a 4 decimales de tasa
icp_clp_cashflow.set_tna_decimal_places(decimales_para_tna)

nuevo_icp_inicio = 10_000.0
icp_clp_cashflow.set_start_date_icp(nuevo_icp_inicio)

nuevo_icp_final = 10_250.0
icp_clp_cashflow.set_end_date_icp(nuevo_icp_final)

print(f"Flujo: {icp_clp_cashflow.amount():,.0f}")

fecha_devengo = qcf.QCDate(29, 3, 2019)
icp_devengo = 10_125.0
tna_devengo = icp_clp_cashflow.get_tna(fecha_devengo, icp_devengo)
print(f"TNA fijada a {fecha_devengo.description(True)}: {tna_devengo:.2%}")
check = round(
    (icp_devengo / nuevo_icp_inicio - 1) * 360.0 / fecha_inicio.day_diff(fecha_devengo),
    decimales_para_tna,
)
print(f"Check: {check:.2%}")
print(f"Interés Devengado al {fecha_devengo.description(True)}: {icp_clp_cashflow.accrued_interest(fecha_devengo, icp_devengo):,.4f}")
print(f"Check: {100 * tna_devengo * fecha_inicio.day_diff(fecha_devengo) / 360.0:,.4f}")

Flujo: 13
TNA fijada a 29-03-2019: 2.37%
Check: 2.37%
Interés Devengado al 29-03-2019: 1.2508
Check: 1.2508


In [113]:
print(qcf.show(icp_clp_cashflow))

('2018-09-20', '2019-09-20', '2019-09-23', 100.0, 10.0, True, 12.504305555555565, 'CLP', 10000.0, 10250.0, 0.0237, 2.504305555555564, 0.0, 1.0, 'LinAct360')


In [114]:
icp_clp_cashflow.get_type()

'IcpClpCashflow'

### Icp Clp Cashflow 2
Un objeto de tipo `IcpClpCashflow2` representa un flujo de caja calculado como un cupón de la pata flotante de un swap ICP (cámara promedio) de Chile. Este tipo de cashflow puede ser *quantizado*, es decir, se puede cambiar su moneda de pago componiéndolo con un objeto adicional. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `float`: spread aditivo a aplicar a la fijación de la TNA
- `float`: spread multiplicativo o gearing a aplicar a la fijación de la TNA
- `float`: el valor del ICP a fecha de inicio (u otro valor arbitrario si el valor es desconocido)
- `float`: el valor del ICP a fecha final (u otro valor arbitrario si el valor es desconocido)

Recordar que TNA significa **Tasa Nominal Anual** y se determina utilizando los valores del índice ICP en la fecha de inicio y fecha final del `IcpClpCashflow`.

In [115]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2018)
fecha_final = qcf.QCDate(20, 9, 2019)
fecha_pago = qcf.QCDate(23, 9, 2019)
nominal = 1_000_000_000.0
amort = 100_000_000.0
spread = 0.0
gearing = 1.0
icp_clp_cashflow2 = qcf.IcpClpCashflow2(
    fecha_inicio,
    fecha_final,
    fecha_pago,
    nominal,
    amort,
    True,
    spread,
    gearing,
    True,
    10_000.0,
    10_250.0,
)

In [116]:
# Getters
print("Fecha Inicio:", icp_clp_cashflow2.get_start_date())
print("Fecha Final:", icp_clp_cashflow2.get_end_date())
print(f"ICP Fecha Inicio: {icp_clp_cashflow2.get_start_date_icp():,.2f}")
print(f"ICP Fecha Final: {icp_clp_cashflow2.get_end_date_icp():,.2f}")

print()
print(f"Valor TNA Todo el Período: {icp_clp_cashflow2.get_rate_value():.2%}")
check = round((10250.0 / 10000 - 1) * 360.0 / fecha_inicio.day_diff(fecha_final), 4)
print(f"Check: {check:.2%}")
print()

print(f"Nominal: {icp_clp_cashflow2.get_nominal():,.0f}")
print(f"Amortización: {icp_clp_cashflow2.get_amortization():,.0f}")
print("Tipo de Tasa:", icp_clp_cashflow2.get_type_of_rate())
print("Moneda:", icp_clp_cashflow2.ccy())

Fecha Inicio: 2018-09-20
Fecha Final: 2019-09-20
ICP Fecha Inicio: 10,000.00
ICP Fecha Final: 10,250.00

Valor TNA Todo el Período: 2.47%
Check: 2.47%

Nominal: 1,000,000,000
Amortización: 100,000,000
Tipo de Tasa: LinAct360
Moneda: CLP


In [117]:
# Setters
decimales_para_tna = 6
icp_clp_cashflow2.set_tna_decimal_places(decimales_para_tna)
print(f"Nueva TNA: {icp_clp_cashflow2.get_rate_value():.4%}")

nuevo_nominal = 100
icp_clp_cashflow2.set_nominal(nuevo_nominal)
print(f"Nuevo Nominal: {icp_clp_cashflow2.get_nominal():,.0f}")

nueva_amortizacion = 10
icp_clp_cashflow2.set_amortization(nueva_amortizacion)
print(f"Nueva Amortización: {icp_clp_cashflow2.get_amortization():,.0f}")

nuevo_icp_inicio = 20_000.0
icp_clp_cashflow2.set_start_date_icp(nuevo_icp_inicio)
print(f"Nuevo ICP Inicio: {icp_clp_cashflow2.get_start_date_icp():,.2f}")

nuevo_icp_final = 20_000.0
icp_clp_cashflow2.set_end_date_icp(nuevo_icp_final)
print(f"Nuevo ICP Final: {icp_clp_cashflow2.get_end_date_icp():,.2f}")
print(f"Check TNA Final: {icp_clp_cashflow2.get_rate_value():.4%}")

Nueva TNA: 2.4658%
Nuevo Nominal: 100
Nueva Amortización: 10
Nuevo ICP Inicio: 20,000.00
Nuevo ICP Final: 20,000.00
Check TNA Final: 0.0000%


In [118]:
# Cálculos
decimales_para_tna = 4  # Se vuelve a 4 decimales de tasa
icp_clp_cashflow2.set_tna_decimal_places(decimales_para_tna)

nuevo_icp_inicio = 10_000.0
icp_clp_cashflow2.set_start_date_icp(nuevo_icp_inicio)

nuevo_icp_final = 10_250.0
icp_clp_cashflow2.set_end_date_icp(nuevo_icp_final)

print(f"Flujo: {icp_clp_cashflow2.amount():,.2f}")
print()

fecha_devengo = qcf.QCDate(29, 3, 2019)
icp_devengo = 10_125.0
tna_devengo = icp_clp_cashflow2.get_tna(fecha_devengo, icp_devengo)
print(f"TNA fijada al {fecha_devengo.description(True)}: {tna_devengo:.2%}")
check = round((icp_devengo / nuevo_icp_inicio - 1) * 360.0 / fecha_inicio.day_diff(fecha_devengo), decimales_para_tna)
print(f"Check: {check:.2%}")
print()

data = qcf.time_series()
data[fecha_devengo] = icp_devengo
print(f"Interés Devengado al {fecha_devengo.description(True)}: {icp_clp_cashflow2.accrued_interest(fecha_devengo, data):,.6f}")
check = 100 * tna_devengo * fecha_inicio.day_diff(fecha_devengo) / 360.0
print(f"Check: {check:,.6f}")

Flujo: 12.50

TNA fijada al 29-03-2019: 2.37%
Check: 2.37%

Interés Devengado al 29-03-2019: 1.250833
Check: 1.250833


In [119]:
print(qcf.show(icp_clp_cashflow2))

('2018-09-20', '2019-09-20', '2019-09-23', 100.0, 10.0, True, 12.499999999999991, 'CLP', 10000.0, 10250.0, 0.0247, 2.504305555555564, 0.0, 1.0, 'LinAct360')


In [120]:
icp_clp_cashflow2.get_type()

'IcpClpCashflow'

### Overnight Index Cashflow
Un objeto de tipo `OvernightIndexCashflow` representa un flujo de caja del tipo de la pata flotante de un swap ICP (cámara promedio) de Chile usando cualquier tipo de índice similar (por ejemplo SOFRINDX) y cualquier moneda. Adicionalmente, permite definir en forma independiente a `start_date` y `end_date` las fechas inicial y final utilizadas para los valores del índice. Esto puede resultar útil cuando una de estas operaciones se utiliza para cubrir créditos o bonos a tasa fija. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha inicio devengo (para la aplicación de la tasa)
- `QCDate`: fecha final devengo (para la aplicación de la tasa)
- `QCDate`: fecha inicio índice (para el valor del índice)
- `QCDate`: fecha final índice (para el valor del índice)
- `QCDate`: fecha de pago
- `QCCurrency`: moneda del nocional
- `float`: nocional (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `float`: spread aditivo a aplicar a la fijación de la tasa equivalente (TNA en el caso de un ICPCLP)
- `float`: spread multiplicativo o gearing a aplicar a la fijación de la tasa equivalente
- `QCInterestRate`: con este objeto se especifica en qué convención se calcula la tasa equivalente
- `string`: nombre del índice overnight a utilizar
- `unsigned int`: número de decimales a utilizar para determinar la tasa equivalente

In [121]:
# Ejemplo
fecha_inicio_devengo = qcf.QCDate(13, 11, 2023)

# Notar que la fecha final de devengo es sábado
fecha_final_devengo = qcf.QCDate(18, 11, 2023)

fecha_inicio_indice = qcf.QCDate(13, 11, 2023)

# Notar que la fecha final de índice es el viernes
fecha_final_indice = qcf.QCDate(17, 11, 2023)

# La fecha de pago es el lunes siguiente
fecha_pago = qcf.QCDate(20, 11, 2023)

moneda_nocional = qcf.QCUSD()
nocional = 1_000_000_000.0
amort = 100_000_000.0
amort_es_flujo = True
spread = 0.0
gearing = 1.0
tasa = qcf.QCInterestRate(0.0, qcf.QCAct360(), qcf.QCLinearWf())
nombre_indice = 'INDICE'
num_decimales = 8
valor_indice_inicio = 1.0
valor_indice_final = 1 + .1234 * 4 / 360 # Suponemos un valor constante de 10% por 4 días del índice

In [122]:
overnight_index_cashflow = qcf.OvernightIndexCashflow(
    fecha_inicio_devengo,
    fecha_final_devengo,
    fecha_inicio_indice,
    fecha_final_indice,
    fecha_pago,
    moneda_nocional,
    nocional,
    amort,
    amort_es_flujo,
    spread,
    gearing,
    tasa,
    nombre_indice,
    num_decimales,
)

#### Getters

In [123]:
print("Fecha Inicio Devengo:", overnight_index_cashflow.get_start_date())
print("Fecha Final Devengo:", overnight_index_cashflow.get_end_date())

print("Fecha Inicio Índice:", overnight_index_cashflow.get_index_start_date())
print("Fecha Final Índice:", overnight_index_cashflow.get_index_end_date())

print("Fecha Pago:", overnight_index_cashflow.get_settlement_date())


print(f"Índice Fecha Inicio: {overnight_index_cashflow.get_start_date_index():,.8f}")
print(f"ïndice Fecha Final: {overnight_index_cashflow.get_end_date_index():,.8f}")

print()
print(f"Valor Tasa Equivalente Todo el Período: {overnight_index_cashflow.get_rate_value():.6%}")
check = round((
    valor_indice_final / valor_indice_inicio - 1
) * 360.0 / fecha_inicio_devengo.day_diff(fecha_final_devengo), num_decimales)
print(f"Check: {check:.6%}")
print()

print(f"Nominal: {overnight_index_cashflow.get_nominal():,.0f}")
print(f"Amortización: {overnight_index_cashflow.get_amortization():,.0f}")
print("Tipo de Tasa:", overnight_index_cashflow.get_type_of_rate())
print("Moneda:", overnight_index_cashflow.ccy())

Fecha Inicio Devengo: 2023-11-13
Fecha Final Devengo: 2023-11-18
Fecha Inicio Índice: 2023-11-13
Fecha Final Índice: 2023-11-17
Fecha Pago: 2023-11-20
Índice Fecha Inicio: 1.00000000
ïndice Fecha Final: 1.00000000

Valor Tasa Equivalente Todo el Período: 0.000000%
Check: 9.872000%

Nominal: 1,000,000,000
Amortización: 100,000,000
Tipo de Tasa: LinAct360
Moneda: USD


#### Setters

In [124]:
decimales_para_tasa_eq = 4
overnight_index_cashflow.set_eq_rate_decimal_places(decimales_para_tasa_eq)
print(f"Nueva Tasa Eq: {overnight_index_cashflow.get_rate_value():.4%}")

Nueva Tasa Eq: 0.0000%


In [125]:
new_notional = 123_456
overnight_index_cashflow.set_nominal(new_notional)
print(f"Nuevo Nocional: {overnight_index_cashflow.get_nominal():,.2f}")

Nuevo Nocional: 123,456.00


In [126]:
new_amortization = 100_000
overnight_index_cashflow.set_amortization(new_amortization)
print(f"Nueva Amortización: {overnight_index_cashflow.get_amortization():,.2f}")

Nueva Amortización: 100,000.00


In [127]:
nuevo_indice_inicio = 20_000.0
overnight_index_cashflow.set_start_date_index(nuevo_indice_inicio)
print(f"Nuevo Índice Inicio: {overnight_index_cashflow.get_start_date_index():,.2f}")

Nuevo Índice Inicio: 20,000.00


In [128]:
nuevo_indice_final = 20_010.0
overnight_index_cashflow.set_end_date_index(nuevo_indice_final)
print(f"Nuevo Índice Final: {overnight_index_cashflow.get_end_date_index():,.2f}")
print(f"Check Tas Eq. Final: {overnight_index_cashflow.get_rate_value():.4%}")

Nuevo Índice Final: 20,010.00
Check Tas Eq. Final: 3.6000%


#### Cálculos

In [129]:
decimales_para_tasa = 8  # Se vuelve a 8 decimales de tasa
overnight_index_cashflow.set_eq_rate_decimal_places(decimales_para_tasa)

nuevo_indice_inicio = 10_000.0
overnight_index_cashflow.set_start_date_index(nuevo_indice_inicio)

nuevo_indice_final = 10_010.0
overnight_index_cashflow.set_end_date_index(nuevo_indice_final)

print(f"Flujo: {overnight_index_cashflow.amount():,.2f}")
print()

Flujo: 100,123.46



In [130]:
fecha_devengo = qcf.QCDate(16, 11, 2023)
indice_devengo = 10_005.0
tasa_devengo = overnight_index_cashflow.get_eq_rate(fecha_devengo, indice_devengo)
print(f"Tasa Eq. fijada al {fecha_devengo.description(True)}: {tasa_devengo:.2%}")
check = round((
    indice_devengo / nuevo_indice_inicio - 1
) * 360.0 / fecha_inicio_devengo.day_diff(fecha_devengo), decimales_para_tasa)
print(f"Check: {check:.2%}")
print()

Tasa Eq. fijada al 16-11-2023: 6.00%
Check: 6.00%



In [131]:
data = qcf.time_series()
data[fecha_devengo] = indice_devengo
print(
    f"Interés Devengado al {fecha_devengo.description(True)}: {overnight_index_cashflow.accrued_interest(fecha_devengo, data):,.6f}")
check = new_notional * tasa_devengo * fecha_inicio_devengo.day_diff(fecha_devengo) / 360.0
print(f"Check: {check:,.6f}")

Interés Devengado al 16-11-2023: 61.728000
Check: 61.728000


In [132]:
overnight_index_cashflow.get_type()

'OvernightIndexCashflow'

#### Show

In [133]:
data = [qcf.show(overnight_index_cashflow)]
pd.DataFrame(data, columns = qcf.get_column_names("OvernightIndexCashflow", ""))

,fecha_inicial_devengo,fecha_final_devengo,fecha_inicial_indice,fecha_final_indice,fecha_pago,nocional,amortizacion,amort_es_flujo,moneda_nocional,nombre_indice,valor_indice_inicial,valor_indice_final,valor_tasa_equivalente,tipo_tasa,interes,flujo,spread,gearing
0,2023-11-13,2023-11-18,2023-11-13,2023-11-17,2023-11-20,123456.0,100000.0,True,USD,INDICE,10000.0,10010.0,0.072,LinAct360,123.456,100123.456,0.0,1.0


### Overnight Index Multi Currency Cashflow

Un objeto de tipo `OvernightIndexMultiCurrencyCashflow` hereda de `OvernightIndexCashflow` y representa un flujo de caja del tipo de la pata flotante de un swap ICP (cámara promedio) de Chile usando cualquier tipo de índice similar (por ejemplo SOFRINDX), cualquier moneda de nocional, pero con flujos de caja en una moneda distinta a la del nocional, por ejemplo un ICPCLP con contraparte en US que compensa en USD. Al heredar de `OvernightIndexCashflow`, también permite definir en forma independiente a `start_date` y `end_date` las fechas inicial y final utilizadas para los valores del índice. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha inicio devengo (para la aplicación de la tasa)
- `QCDate`: fecha final devengo (para la aplicación de la tasa)
- `QCDate`: fecha inicio índice (para el valor del índice)
- `QCDate`: fecha final índice (para el valor del índice)
- `QCDate`: fecha de pago
- `QCCurrency`: moneda del nocional
- `float`: nocional (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `float`: spread aditivo a aplicar a la fijación de la tasa equivalente (TNA en el caso de un ICPCLP)
- `float`: spread multiplicativo o gearing a aplicar a la fijación de la tasa equivalente
- `QCInterestRate`: con este objeto se especifica en qué convención se calcula la tasa equivalente
- `string`: nombre del índice overnight a utilizar
- `unsigned int`: número de decimales a utilizar para determinar la tasa equivalente
  
Hasta acá son los mismos argumentos necesarios para construir un `OvernightIndexCAshflow`. Se añaden los siguientes argumentos:
- `QCDate`: fecha de fixing del índice de tipo de cambio. Esta fecha se refiere a la fecha de publicación del índice, no a la fecha de fixing en sentido financiero.
- `QCCurrency`: moneda de pago de los flujos de caja
- `FXRateIndex>`: índice de tipo de cambio utilizado para la conversión de los flujos a moneda de pago

#### Ejemplo

In [134]:
fecha_inicio_devengo = qcf.QCDate(13, 11, 2023)

# Notar que la fecha final de devengo es sábado
fecha_final_devengo = qcf.QCDate(18, 11, 2023)

fecha_inicio_indice = qcf.QCDate(13, 11, 2023)

# Notar que la fecha final de índice es el viernes
fecha_final_indice = qcf.QCDate(17, 11, 2023)

# La fecha de pago es el lunes siguiente
fecha_pago = qcf.QCDate(20, 11, 2023)

moneda_nocional = qcf.QCUSD()
nocional = 1_000_000_000.0
amort = 100_000_000.0
amort_es_flujo = True
spread = 0.0
gearing = 1.0
tasa = qcf.QCInterestRate(0.0, qcf.QCAct360(), qcf.QCLinearWf())
nombre_indice = 'INDICE'
num_decimales = 8
valor_indice_inicio = 1.0
valor_indice_final = 1 + .1234 * 4 / 360 # Suponemos un valor constante de 10% por 4 días del índice

# -------------------------------
fecha_fixing_fx_index = fecha_final_devengo
moneda_pago = qcf.QCCLP()
indice_fx = usdclp_obs

In [135]:
overnight_index_mccy_cashflow = qcf.OvernightIndexMultiCurrencyCashflow(
    fecha_inicio_devengo,
    fecha_final_devengo,
    fecha_inicio_indice,
    fecha_final_indice,
    fecha_pago,
    moneda_nocional,
    nocional,
    amort,
    amort_es_flujo,
    spread,
    gearing,
    tasa,
    nombre_indice,
    num_decimales,
    fecha_fixing_fx_index,
    moneda_pago,
    indice_fx,
)

In [136]:
type(overnight_index_mccy_cashflow)

qcfinancial.OvernightIndexMultiCurrencyCashflow

#### Función `show` 

In [137]:
qcf.show(overnight_index_mccy_cashflow)

('2023-11-13',
 '2023-11-18',
 '2023-11-13',
 '2023-11-17',
 '2023-11-20',
 1000000000.0,
 100000000.0,
 True,
 'USD',
 'INDICE',
 1.0,
 1.0,
 0.0,
 'LinAct360',
 0.0,
 100000000.0,
 0.0,
 1.0,
 'CLP',
 'USDOBS',
 '2023-11-18',
 1.0,
 0.0,
 100000000.0,
 100000000.0)

Se envuelve el resultado en un `pd.DataFrame`.

In [138]:
pd.DataFrame(
    [qcf.show(overnight_index_mccy_cashflow),],
    columns=qcf.get_column_names("OvernightIndexMultiCurrencyCashflow", "")
)

,fecha_inicial_devengo,fecha_final_devengo,fecha_inicial_indice,fecha_final_indice,fecha_pago,nocional,amortizacion,amort_es_flujo,moneda_nocional,nombre_indice,...,flujo,spread,gearing,moneda_pago,indice_fx,fecha_fijacion_indice_fx,valor_indice_fx,interes_moneda_pago,amortizacion_moneda_pago,flujo_moneda_pago
0,2023-11-13,2023-11-18,2023-11-13,2023-11-17,2023-11-20,1.000000e+09,100000000.0,True,USD,INDICE,...,100000000.0,0.0,1.0,CLP,USDOBS,2023-11-18,1.0,0.0,100000000.0,100000000.0


#### Nuevo Setter

Valor del índice de tipo de cambio.

In [139]:
overnight_index_mccy_cashflow.set_fx_rate_index_value(.5)

Ver el efecto en las últimas dos columnas.

In [140]:
pd.DataFrame(
    [qcf.show(overnight_index_mccy_cashflow),],
    columns=qcf.get_column_names("OvernightIndexMultiCurrencyCashflow", "")
)

,fecha_inicial_devengo,fecha_final_devengo,fecha_inicial_indice,fecha_final_indice,fecha_pago,nocional,amortizacion,amort_es_flujo,moneda_nocional,nombre_indice,...,flujo,spread,gearing,moneda_pago,indice_fx,fecha_fijacion_indice_fx,valor_indice_fx,interes_moneda_pago,amortizacion_moneda_pago,flujo_moneda_pago
0,2023-11-13,2023-11-18,2023-11-13,2023-11-17,2023-11-20,1.000000e+09,100000000.0,True,USD,INDICE,...,100000000.0,0.0,1.0,CLP,USDOBS,2023-11-18,0.5,0.0,50000000.0,50000000.0


#### Nuevos Getters

In [141]:
overnight_index_mccy_cashflow.get_fx_rate_index()

In [142]:
overnight_index_mccy_cashflow.get_type()

'OvernightIndexMultiCurrencyCashflow'

In [143]:
overnight_index_mccy_cashflow.get_fx_rate_index_code()

'USDOBS'

In [144]:
overnight_index_mccy_cashflow.get_fx_rate_index_value()

0.5

#### Nuevos Cálculos

Primero se fijan los valores del índice overnight.

In [145]:
overnight_index_mccy_cashflow.set_start_date_index(100)
overnight_index_mccy_cashflow.accrued_interest(fecha_final_devengo, 102)

20000000.00000002

In [146]:
overnight_index_mccy_cashflow.settlement_ccy_interest()

-495000000.0

In [147]:
overnight_index_mccy_cashflow.settlement_ccy_amortization()

50000000.0

In [148]:
overnight_index_mccy_cashflow.settlement_ccy_amount()

-445000000.0

### Icp Clf Cashflow
Un objeto de tipo `IcpClfCashflow` representa un flujo de caja calculado como un cupón de la pata flotante de un swap ICP (cámara promedio) en UF de Chile. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `float`: spread aditivo a aplicar a la fijación de la TRA
- `float`: spread multiplicativo o gearing a aplicar a la fijación de la TRA
- `vector<float>`: objeto `double_vec` (en Python) que contiene ICP Inicio, ICP Final, UF Inicio, UF Final (se debe respetar el orden)

Recordar que TRA significa **Tasa Real Anual** y se determina utilizando los valores del índice ICP y los valores de la UF en la fecha de inicio y fecha final del `IcpClfCashflow`.

In [149]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2018)
fecha_final = qcf.QCDate(20, 9, 2019)
fecha_pago = qcf.QCDate(23, 9, 2019)
nominal = 300_000.0
amort = 100_000.0
spread = 0.0
gearing = 1.0

icp_uf = qcf.double_vec()
# Los primeros dos valores corresponden a icp_inicial e icp_final.
# Los segundos dos valores corresponden a uf_inicial y uf_final
icp_uf.append(10_000.0)
icp_uf.append(10_250.0)
icp_uf.append(35_000.0)
icp_uf.append(35_500.0)

icp_clf_cashflow = qcf.IcpClfCashflow(
    fecha_inicio, 
    fecha_final, 
    fecha_pago, 
    nominal, 
    amort, 
    True, 
    spread, 
    gearing, 
    icp_uf
)

In [150]:
# Getters
print("Fecha Inicio:", icp_clf_cashflow.get_start_date())
print("Fecha Final:", icp_clf_cashflow.get_end_date())

print(f"ICP Fecha Inicio: {icp_clf_cashflow.get_start_date_icp():,.2f}")
print(f"ICP Fecha Final: {icp_clf_cashflow.get_end_date_icp():,.2f}")

print(f"UF Fecha Inicio: {icp_clf_cashflow.get_start_date_uf():,.2f}")
print(f"UF Fecha Final: {icp_clf_cashflow.get_end_date_uf():,.2f}")

print(f"Valor TRA Todo el Período: {icp_clf_cashflow.get_rate_value():.4%}")
tna = icp_clf_cashflow.get_tna(fecha_final, 10_250.0)
dias = fecha_inicio.day_diff(fecha_final)
tra = ((1 + tna * dias / 360.0) * 35_000.0 / 35_500.0 - 1) * 360.0 / dias
print(f"Check TRA: {round(tra, 6):.4%}")

print(f"Nominal: {icp_clf_cashflow.get_nominal():,.0f}")
print(f"Amortización: {icp_clf_cashflow.get_amortization():,.0f}")
print("Tipo de Tasa:", icp_clf_cashflow.get_type_of_rate())
print("Moneda:", icp_clf_cashflow.ccy())

Fecha Inicio: 2018-09-20
Fecha Final: 2019-09-20
ICP Fecha Inicio: 10,000.00
ICP Fecha Final: 10,250.00
UF Fecha Inicio: 35,000.00
UF Fecha Final: 35,500.00
Valor TRA Todo el Período: 1.0461%
Check TRA: 1.0461%
Nominal: 300,000
Amortización: 100,000
Tipo de Tasa: LinAct360
Moneda: CLF


In [151]:
# Setters
decimales_para_tra = 8
icp_clf_cashflow.set_tra_decimal_places(decimales_para_tra)
print(f"Nueva TRA: {icp_clf_cashflow.get_rate_value():.6%}")

nuevo_nominal = 100_000.0
icp_clf_cashflow.set_nominal(nuevo_nominal)
print(f"Nuevo Nominal: {icp_clf_cashflow.get_nominal():,.0f}")

nueva_amortizacion = 10_000.0
icp_clf_cashflow.set_amortization(nueva_amortizacion)
print(f"Nueva Amortización: {icp_clf_cashflow.get_amortization():,.0f}")

nuevo_icp_inicio = 20_000.0
icp_clf_cashflow.set_start_date_icp(nuevo_icp_inicio)
print(f"Nuevo ICP Inicio: {icp_clf_cashflow.get_start_date_icp():,.2f}")

nuevo_icp_final = 20_500.0
icp_clf_cashflow.set_end_date_icp(nuevo_icp_final)
print(f"Nuevo ICP Final: {icp_clf_cashflow.get_end_date_icp():,.2f}")
print(f"Check TNA Final: {icp_clf_cashflow.get_tna(fecha_final, nuevo_icp_final):.6%}")

Nueva TRA: 1.046054%
Nuevo Nominal: 100,000
Nueva Amortización: 10,000
Nuevo ICP Inicio: 20,000.00
Nuevo ICP Final: 20,500.00
Check TNA Final: 2.470000%


In [152]:
icp_clf_cashflow.get_type()

'IcpClfCashflow'

In [153]:
qcf.show(icp_clf_cashflow)

('2018-09-20',
 '2019-09-20',
 '2019-09-23',
 100000.0,
 10000.0,
 True,
 11060.582527777788,
 'CLF',
 20000.0,
 20500.0,
 35000.0,
 35500.0,
 0.01046054,
 1060.582527777787,
 0.0,
 1.0,
 'LinAct360',
 392650680.0)

### Compounded Overnight Rate Cashflow

Un objeto de tipo `CompoundedOvernightRateCashflow` representa un flujo de caja calculado como un cupón de la pata flotante de un OIS sobre cualquier índice overnight (SOFR, FF, EONIA). Para dar de alta uno de estos objetos se requiere:

- `InterestRateIndex`: el índice de tasa de interés prefijado
- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de pago
- `DateList`: fechas de fixing del índice
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `QCCurrency`: moneda del nocional de la operación
- `float`: spread aditivo a aplicar a la fijación de la TRA
- `float`: spread multiplicativo o gearing a aplicar a la fijación de la TRA
- `bool`: si `True` la tasa equivalente se calcula en convención Lin Act/360. En caso contrario es Lin 30/360
- `unsigned int`: número de decimales a usar en el cálculo de la tasa equivalente. Por ejemplo para 0.12345678% este valor debe ser 10.
- `unsigned int`: número de días de lookback
- `unsigned int`: número de días de lockout

El efecto de lookback y lockout aún no está implementado.

#### Constructor

Se da de alta un índice overnight ficiticio de test.

In [154]:
codigo = "OITEST"
lin_act360 = qcf.QCInterestRate(0.0, act360, lin_wf)
fixing_lag = qcf.Tenor("0d")
tenor = qcf.Tenor("1d")
fixing_calendar = scl
settlement_calendar = scl
oitest = qcf.InterestRateIndex(
    codigo, 
    lin_act360, 
    fixing_lag, 
    tenor, 
    fixing_calendar, 
    settlement_calendar, 
    usd
)

In [155]:
fixing_dates = qcf.DateList()

In [156]:
fixing_dates.append(qcf.QCDate(27, 12, 2021))
fixing_dates.append(qcf.QCDate(28, 12, 2021))
fixing_dates.append(qcf.QCDate(29, 12, 2021))
fixing_dates.append(qcf.QCDate(30, 12, 2021))

In [157]:
cor_cashflow = qcf.CompoundedOvernightRateCashflow(
    oitest,
    qcf.QCDate(27, 12, 2021),
    qcf.QCDate(31, 12, 2021),
    qcf.QCDate(31, 12, 2021),
    fixing_dates,
    10_000_000.0,
    100_000.0,
    True,
    qcf.QCCLP(),
    spread:=0.001,
    1.0,
    True,
    8,
    0,
    0,
)

#### Getters

In [158]:
cor_cashflow.get_start_date().description(False)

'2021-12-27'

In [159]:
cor_cashflow.get_end_date().description(False)

'2021-12-31'

In [160]:
cor_cashflow.get_settlement_date().description(False)

'2021-12-31'

In [161]:
for d in cor_cashflow.get_fixing_dates():
    print(d)

2021-12-27
2021-12-28
2021-12-29
2021-12-30


In [162]:
print(f"Nominal: {cor_cashflow.get_nominal():,.0f}")

Nominal: 10,000,000


In [163]:
print(f"Amortization: {cor_cashflow.get_amortization():,.0f}")

Amortization: 100,000


In [164]:
cor_cashflow.get_initial_currency().get_iso_code()

'CLP'

In [165]:
print(f"Spread: {cor_cashflow.get_spread():.2%}")

Spread: 0.10%


In [166]:
cor_cashflow.get_gearing()

1.0

In [167]:
cor_cashflow.get_type()

'CompoundedOvernightRateCashflow'

In [168]:
cor_cashflow.get_eq_rate_decimal_places()

8

In [169]:
derivs = cor_cashflow.get_amount_derivatives()

In [170]:
len(derivs)

2

In [171]:
for der in derivs:
    print(der)

0.0
0.0


#### Setters

In [172]:
cor_cashflow.set_nominal(1_000)
print(f"Nominal: {cor_cashflow.get_nominal():,.0f}")

Nominal: 1,000


In [173]:
cor_cashflow.set_amortization(0)
cor_cashflow.get_amortization()

0.0

Se reversa el ejemplo.

In [174]:
cor_cashflow.set_nominal(10_000_000.0)
cor_cashflow.set_amortization(100_000.0)

#### Accrued Fixing

Para el cálculo de `accrued_fixing` se requiere un objeto de tipo `TimeSeries` que contenga los datos históricos del índice overnight.

In [175]:
cor_cashflow.accrued_fixing(qcf.QCDate(29, 12, 2021))

ValueError: A TimeSeries object with overnight rate values is needed.

In [ ]:
ts = qcf.time_series()

In [ ]:
ts[qcf.QCDate(27, 12, 2021)] = 0.01
ts[qcf.QCDate(28, 12, 2021)] = 0.02
ts[qcf.QCDate(29, 12, 2021)] = 0.03
ts[qcf.QCDate(30, 12, 2021)] = 0.04

In [ ]:
print(f"Accrued fixing: {cor_cashflow.accrued_fixing(qcf.QCDate(29, 12, 2021), ts):.6%}")

In [ ]:
check = ((1 + 0.01 / 360) * (1 + 0.02 / 360.0) - 1) * 360 / 2.0
print(f"Check: {check:.6%}")

#### Accrued Interest

Para el cálculo de `accrued_interest` se requiere un objeto de tipo `TimeSeries` que contenga los datos históricos del índice overnight.

In [ ]:
cor_cashflow.accrued_interest(qcf.QCDate(29, 12, 2021))

In [ ]:
print(f"Accrued interest: {cor_cashflow.accrued_interest(qcf.QCDate(29, 12, 2021), ts):,.2f}")

In [ ]:
check = (
    cor_cashflow.get_nominal()
    * (cor_cashflow.accrued_fixing(qcf.QCDate(29, 12, 2021), ts) + spread)
    * 2
    / 360.0
)
print(f"Check: {check:,.2f}")

In [ ]:
print(f"Amount: {cor_cashflow.amount():,.2f}")

In [ ]:
cor_cashflow.date().description(False)

In [ ]:
cor_cashflow.fixing(ts)

In [ ]:
print(f"Interest: {cor_cashflow.interest():,.2f}")

In [ ]:
cor_cashflow.is_expired(qcf.QCDate(29, 12, 2021))

In [ ]:
cor_cashflow.is_expired(qcf.QCDate(31, 12, 2021))

In [ ]:
cor_cashflow.is_expired(qcf.QCDate(1, 1, 2022))

In [ ]:
cor_cashflow.get_type()

In [ ]:
qcf.show(cor_cashflow)

### Compounded Overnight Rate Cashflow 2

Un objeto de tipo `CompoundedOvernightRateCashflow2` representa un flujo de caja calculado como un cupón de la pata flotante de un OIS sobre cualquier índice overnight (SOFR, FF, EONIA). Para dar de alta uno de estos objetos se requiere:

- `InterestRateIndex`: el índice de tasa de interés prefijado
- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de pago
- `DateList`: fechas de fixing del índice
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `QCCurrency`: moneda del nocional de la operación
- `float`: spread aditivo a aplicar a la fijación de la TRA
- `float`: spread multiplicativo o gearing a aplicar a la fijación de la TRA
- `QCInterestRate`: permite especificar cuál es la convención de cálculo de la tasa equivalente
- `unsigned int`: número de decimales a usar en el cálculo de la tasa equivalente. Por ejemplo para 0.12345678% este valor debe ser 10.
- `unsigned int`: número de días de lookback
- `unsigned int`: número de días de lockout

El efecto de lookback y lockout aún no está implementado.

#### Constructor

Se da de alta un índice overnight ficiticio de test.

In [ ]:
codigo = "OITEST"
lin_act360 = qcf.QCInterestRate(0.0, act360, lin_wf)
fixing_lag = qcf.Tenor("0d")
tenor = qcf.Tenor("1d")
fixing_calendar = scl
settlement_calendar = scl
oitest = qcf.InterestRateIndex(
    codigo, 
    lin_act360, 
    fixing_lag, 
    tenor, 
    fixing_calendar, 
    settlement_calendar, 
    usd
)

In [ ]:
fixing_dates = qcf.DateList()

In [ ]:
fixing_dates.append(qcf.QCDate(27, 12, 2021))
fixing_dates.append(qcf.QCDate(28, 12, 2021))
fixing_dates.append(qcf.QCDate(29, 12, 2021))
fixing_dates.append(qcf.QCDate(30, 12, 2021))

In [ ]:
cor_cashflow_2 = qcf.CompoundedOvernightRateCashflow2(
    oitest,
    qcf.QCDate(27, 12, 2021),
    qcf.QCDate(31, 12, 2021),
    qcf.QCDate(31, 12, 2021),
    fixing_dates,
    10_000_000.0,
    100_000.0,
    True,
    qcf.QCCLP(),
    spread:=0.001,
    1.0,
    qcf.QCInterestRate(0.0, qcf.QCAct360(), qcf.QCLinearWf()),
    8,
    0,
    0,
)

#### Getters

In [ ]:
cor_cashflow_2.get_start_date().description(False)

In [ ]:
cor_cashflow_2.get_end_date().description(False)

In [ ]:
cor_cashflow_2.get_settlement_date().description(False)

In [ ]:
for d in cor_cashflow_2.get_fixing_dates():
    print(d)

In [ ]:
print(f"Nocional: {cor_cashflow_2.get_nominal():,.0f}")

In [ ]:
print(f"Amortization: {cor_cashflow_2.get_amortization():,.0f}")

In [ ]:
cor_cashflow_2.ccy().get_iso_code()

In [ ]:
print(f"Spread: {cor_cashflow_2.get_spread():.2%}")

In [ ]:
cor_cashflow_2.get_gearing()

In [ ]:
cor_cashflow_2.get_type()

In [ ]:
cor_cashflow_2.get_eq_rate_decimal_places()

In [ ]:
derivs = cor_cashflow_2.get_amount_derivatives()

In [ ]:
len(derivs)

In [ ]:
for der in derivs:
    print(der)

#### Setters

In [ ]:
cor_cashflow_2.set_notional(1_000)
print(f"Nocional: {cor_cashflow_2.get_nominal():,.0f}")

In [ ]:
cor_cashflow_2.set_amortization(0)
cor_cashflow_2.get_amortization()

Se reversa el ejemplo.

In [ ]:
cor_cashflow_2.set_notional(10_000_000.0)
cor_cashflow_2.set_amortization(100_000.0)

#### Accrued Fixing

Para el cálculo de `accrued_fixing` se requiere un objeto de tipo `TimeSeries` que contenga los datos históricos del índice overnight.

In [ ]:
ts = qcf.time_series()

In [ ]:
ts[qcf.QCDate(27, 12, 2021)] = 0.01
ts[qcf.QCDate(28, 12, 2021)] = 0.02
ts[qcf.QCDate(29, 12, 2021)] = 0.03
ts[qcf.QCDate(30, 12, 2021)] = 0.04

In [ ]:
print(f"Accrued fixing: {cor_cashflow_2.accrued_fixing(qcf.QCDate(29, 12, 2021), ts):.6%}")

In [ ]:
check = ((1 + 0.01 / 360) * (1 + 0.02 / 360.0) - 1) * 360 / 2.0
print(f"Check: {check:.6%}")

#### Accrued Interest

Para el cálculo de `accrued_interest` se requiere un objeto de tipo `TimeSeries` que contenga los datos históricos del índice overnight.

In [ ]:
print(f"Accrued interest: {cor_cashflow_2.accrued_interest(qcf.QCDate(29, 12, 2021), ts):,.2f}")

In [ ]:
check = (
    cor_cashflow_2.get_nominal()
    * (cor_cashflow_2.accrued_fixing(qcf.QCDate(29, 12, 2021), ts) + spread)
    * 2
    / 360.0
)
print(f"Check: {check:,.2f}")

In [ ]:
print(f"Amount: {cor_cashflow_2.amount():,.2f}")

In [ ]:
cor_cashflow_2.date().description(False)

In [ ]:
print(f"Interest: {cor_cashflow_2.interest_from_spread():,.2f}")

In [ ]:
cor_cashflow_2.is_expired(qcf.QCDate(29, 12, 2021))

In [ ]:
cor_cashflow_2.is_expired(qcf.QCDate(31, 12, 2021))

In [ ]:
cor_cashflow_2.is_expired(qcf.QCDate(1, 1, 2022))

In [ ]:
cor_cashflow_2.get_type()

In [ ]:
qcf.show(cor_cashflow)

In [ ]:
pd.DataFrame([qcf.show(cor_cashflow)], columns=qcf.get_column_names("CompoundedOvernightRateCashflow2", ""))

### Compounded Overnight Rate Multi Currency Cashflow 2

Un objeto de tipo `CompoundedOvernightRateMultiCurrencyCashflow2` representa un flujo de caja calculado como un cupón de la pata flotante de un OIS sobre cualquier índice overnight (SOFR, FF, EONIA) que se liquidará en una moneda distinta de la moneda del nominal utilizando el valor a una cierta fecha de un índice de tipo de cambio prefijado.. Para dar de alta uno de estos objetos se requiere:

- `InterestRateIndex`: el índice de tasa de interés prefijado
- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de pago
- `DateList`: fechas de fixing del índice
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `QCCurrency`: moneda del nocional de la operación
- `float`: spread aditivo a aplicar a la fijación de la TRA
- `float`: spread multiplicativo o gearing a aplicar a la fijación de la TRA
- `QCInterestRate`: permite especificar cuál es la convención de cálculo de la tasa equivalente
- `unsigned int`: número de decimales a usar en el cálculo de la tasa equivalente. Por ejemplo para 0.12345678% este valor debe ser 10.
- `unsigned int`: número de días de lookback
- `unsigned int`: número de días de lockout
- `QCDate`: fecha de fixing del índice FX
- `QCCurrency`: moneda de pago o settlement
- `FXRateIndex`: índice de tipo de cambio a utilizar

El efecto de lookback y lockout aún no está implementado.

#### Constructor

Se da de alta un índice overnight ficiticio de test.

In [ ]:
codigo = "OITEST"
lin_act360 = qcf.QCInterestRate(0.0, act360, lin_wf)
fixing_lag = qcf.Tenor("0d")
tenor = qcf.Tenor("1d")
fixing_calendar = scl
settlement_calendar = scl
oitest = qcf.InterestRateIndex(
    codigo, 
    lin_act360, 
    fixing_lag, 
    tenor, 
    fixing_calendar, 
    settlement_calendar, 
    usd
)

In [ ]:
fixing_dates = qcf.DateList()

In [ ]:
fixing_dates.append(qcf.QCDate(27, 12, 2021))
fixing_dates.append(qcf.QCDate(28, 12, 2021))
fixing_dates.append(qcf.QCDate(29, 12, 2021))
fixing_dates.append(qcf.QCDate(30, 12, 2021))

In [ ]:
cor_cashflow_mccy_2 = qcf.CompoundedOvernightRateMultiCurrencyCashflow2(
    oitest,
    qcf.QCDate(27, 12, 2021),
    qcf.QCDate(31, 12, 2021),
    qcf.QCDate(31, 12, 2021),
    fixing_dates,
    10_000_000.0,
    100_000.0,
    True,
    qcf.QCCLP(),
    spread:=0.001,
    1.0,
    qcf.QCInterestRate(0.0, qcf.QCAct360(), qcf.QCLinearWf()),
    8,
    0,
    0,
    qcf.QCDate(31, 12, 2021),
    qcf.QCUSD(),
    usdclp_obs,
)

In [ ]:
qcf.show(cor_cashflow_mccy_2)

In [ ]:
pd.DataFrame([qcf.show(cor_cashflow_mccy_2)], columns=qcf.get_column_names("CompoundedOvernightRateMultiCurrencyCashflow2", ""))

In [ ]:
cor_cashflow_mccy_2.set_fx_rate_index_value(2.0)

In [ ]:
cor_cashflow_mccy_2.interest(ts)

In [ ]:
cor_cashflow_mccy_2.to_settlement_currency(cor_cashflow_mccy_2.interest(ts))